# Find the position of the obstacle x_predict, y_predict for each condition (72 in total)
# Update: adding varying x into modeling. Previous: fixed x but varying y.

# (1) Pure Linear Projection

A **scatter plot of path projection predictions vs. human collider placements**, as a baseline to compare a more sophisticated model that uses intuitive physics.

## Load Dataset

In [1]:
import pandas as pd
import statsmodels.api as sm
import os
import json
import ast
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.linear_model import LinearRegression
from scipy.stats import binned_statistic_2d
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib.colorbar import ColorbarBase
from matplotlib.cm import hsv
from shapely.geometry import Point, LineString
import pybrms
from similaritymeasures import frechet_dist
from scipy.stats import pearsonr
from scipy.stats import norm
from scipy.stats import multivariate_normal
import pickle
import subprocess
from scipy.optimize import minimize, Bounds
from pathlib import Path
from matplotlib.animation import FuncAnimation

canvasWidth = 1000 
canvasHeight = 600
ball_Xs = [322.3, 322.3, 322.3, 322.3, 500 , 500 , 500 , 500 , 604.4, 604.4, 604.4, 604.4]
color_map = plt.get_cmap('viridis', 6)
green = color_map(4)
modeling_choice = 'Model_both_x_and_y'

In [2]:
# Define colors for different elements
ground_truth_color = 'cyan'
model_prediction_color = 'darkblue'
# human_centroid_color = 'gray'
human_centroid_color = '#0A704E'
participant_color = green 
# participant_color = 'blueviolet'
screen_color = 'lightgray'
ball_color = '#ececd1'
# anchor_color = '#f59f01'
# anchor_color = '#0A704E' # dark_green
anchor_color = 'darkviolet'

In [3]:
# change the string in a dataframe to list when loading csv
def string2List(dataString):
    dataList = ast.literal_eval(dataString)
    return dataList

In [4]:
# given the summed log likelihoods, sample size, and the number of free parameters, calculate BIC
# sample size should be the number of observations/the number of data points
def getBIC(k, sampleSize, sumLogLike):
    result = k*np.log(sampleSize) - 2*sumLogLike
    return result

## Add fd_metric to the dataframe

In [5]:
# canvas settings
segmented_line = [((250, 150), (250, 450)), ((250, 450), (750, 450)), ((750, 450), (750, 150))]
top_y = 150
bottom_y = 450
left_x = 150
right_x = 750
ball_radius = 30
obstacle_radius = 45

In [6]:
# select all points that are outside of the screen
# return list1, list2
# list1: list of points that form the path before the ball enters the screen (when ball_y <= top_y + ball_radius)
# list2: list of points that form the path before the ball exits the screen (when ball_y >= bottom_y - ball_radius or ball_x <= left_x + ball_radius or ball_x >= right_x - ball_radius)
def pickPointsOutScreen(path, top_y, bottom_y, left_x, right_x, ball_radius, keepSpeed = False):
    list1 = []
    list2 = []
    if len(path[0])==3:
        for ball_x,ball_y,speed in path:
            if ball_y <= top_y + ball_radius:
                if keepSpeed:
                    list1.append((ball_x,ball_y,speed))
                else:
                    list1.append((ball_x,ball_y))
            if (ball_y >= bottom_y - ball_radius) or (ball_x <= left_x + ball_radius) or (ball_x >= right_x - ball_radius):
                if keepSpeed:
                    list2.append((ball_x,ball_y,speed))
                else:
                    list2.append((ball_x,ball_y))
    else: 
        for ball_x,ball_y in path:
            if ball_y <= top_y + ball_radius:
                list1.append((ball_x,ball_y))
            if (ball_y >= bottom_y - ball_radius) or (ball_x <= left_x + ball_radius) or (ball_x >= right_x - ball_radius):
                list2.append((ball_x,ball_y))
    return list1, list2

# calculate Fréchet distance between list1_participant and list1_groundTruth (fd1) and list2_participant and list2_groundTruth (fd2)
# return fd1, fd2
def evaluateResponseByFD(rowData):
    simulatedPath = rowData['simulated_path']
    exactPath = rowData['exact_path_single']
    l1_p, l2_p = pickPointsOutScreen(simulatedPath, top_y, bottom_y, left_x, right_x, ball_radius)
    l1_g, l2_g = pickPointsOutScreen(exactPath, top_y, bottom_y, left_x, right_x, ball_radius)
    fd1 = frechet_dist(l1_p, l1_g)
    fd2 = frechet_dist(l2_p, l2_g)
    return fd1, fd2

## Load Full Data Stored from Previous Analysis

In [7]:
# this dataframe contains the ground truth trajectories of the ball, simulated trajectory of each participant's response, as well as the fd1, fd2 evaluated 
allData = pd.read_csv('df_all_with_single_ground_trajectory_simulation_fd_pilot_v1.csv')
allData['exact_path_single'] = allData['exact_path_single'].apply(string2List)
allData['simulated_path'] = allData['simulated_path'].apply(string2List)
allData['fd_combined'] = allData['fd1_enter'] + allData['fd2_exit']

In [ ]:
print(allData)

In [ ]:
import seaborn as sns

# Visualizing the fd_combined by conditions

# Create a boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='stimulus_idx', y='fd_combined', data=allData, width = 0.5, flierprops={"marker": "o", "markersize": 3},showmeans=True,  
            meanprops={'marker':'o',
                       'markerfacecolor':'white', 
                       'markeredgecolor':'black',
                       'markersize':'5'},
            hue='stimulus_idx',  # Assign the 'x' variable also to 'hue'
            palette='Set2',  # Use a qualitative color palette
            legend=False) 

# Add a horizontal line at y=0
plt.axhline(y=0, color='gray', linestyle='--')

# Adding labels and title
plt.xlabel('Condition')
plt.ylabel('Similarity')
# plt.title('Distribution of Fréchet Distances Across Conditions')

# Show the plot
# plt.savefig('fd_12_conditions.pdf')
plt.show()

In [ ]:
# mean of the fd performances
allData['fd_combined'].mean() # 77.746

## Get Linear Projection Predictions

In [11]:
segmented_line = [((250, 150), (250, 450)), ((250, 450), (750, 450)), ((750, 450), (750, 150))]

# Create a function that find the two consecutive coordinates of the ball when it is about to fall outside of the screen
def distance_to_segment(point, segment):
    """Calculate the distance from a point to a line segment."""
    return Point(point).distance(LineString(segment))

def closest_points_to_line(points, segmented_line = segmented_line):
    """Find two points that are nearest to the segmented line and are also near each other."""
    # Calculate distances from all points to the segmented line
    distances = [min(distance_to_segment(point, segment) for segment in segmented_line) for point in points]
    
    # Sort points based on their distance to the segmented line
    sorted_points = sorted(points, key=lambda p: min(distance_to_segment(p, segment) for segment in segmented_line))
    
    for point in sorted_points:
        # Sort other points based on their distance to the current point
        other_points = sorted(points, key=lambda p: np.linalg.norm(np.array(point) - np.array(p)))
        for near_point in other_points[1:3]:  # considering the 2 nearest points
            if abs(distances[points.index(point)] - distances[points.index(near_point)]) < 5:
                return point, near_point
    return None

# Function to get the first two elements of each tuple
def take_first_two(lst):
    return [(a, b) for a, b, _ in lst]

In [12]:
# create the no speed path col
allData['exact_path_no_speed'] = allData['exact_path_single'].apply(take_first_two)

# create the vertical line col
allData['projected_line_0'] = allData['ball_X']

# create the tangential line col
allData['projected_line_1'] = allData['exact_path_no_speed'].apply(closest_points_to_line)

## save allData to pickle to skip some of the data preparation steps

In [8]:
# Save the DataFrame as a Pickle file
# allData.to_pickle('allData_Sep_29.pkl')

# Load it back without needing string2List
allData = pd.read_pickle('allData_Sep_29.pkl')

### check by visualizing

In [12]:
def drawProjectedPath(condition, merged_trajectory, ax, zorder):
    # Given data
    subset = merged_trajectory[merged_trajectory['stimulus_idx']==condition]
    x_vertical = subset['projected_line_0'].iloc[0]
    point1 = subset['projected_line_1'].iloc[0][0]
    point2 = subset['projected_line_1'].iloc[0][1]
    
    # Calculate slope
    if (point2[0] - point1[0]) == 0:
        slope = float('inf')
    else:
        slope = (point2[1] - point1[1]) / (point2[0] - point1[0])
        
        
    # Calculate extended points based on the slope
    x_extended_1 = 100  # at the far left of the plot
    y_extended_1 = point1[1] + slope * (x_extended_1 - point1[0])

    x_extended_2 = 900  # at the far right of the plot
    y_extended_2 = point1[1] + slope * (x_extended_2 - point1[0])
    
    # Create the two line segments
    vertical_line = LineString([(x_vertical, -200), (x_vertical, 600)])  # assuming a large y range for the vertical line
    extended_line = LineString([(x_extended_1, y_extended_1), (x_extended_2, y_extended_2)])
    
    # Calculate intersection
    intersection = vertical_line.intersection(extended_line)
    print(f'stimulus:{condition}')
    print(intersection)

    # Plotting
    if intersection and 100 <= intersection.x <= 900 and 60 <= intersection.y <= 540:
        ax.plot([x_vertical, x_vertical], [0, intersection.y], label="Vertical Line", linestyle='--', color='black', zorder = zorder) 
        if intersection.y <= y_extended_1:
            ax.plot([x_extended_1, intersection.x], [y_extended_1, intersection.y], label="Extended Line", linestyle='--', color='black', zorder = zorder)
        else:
            ax.plot([intersection.x, x_extended_2], [intersection.y, y_extended_2], label="Extended Line", linestyle='--', color='black', zorder = zorder)
#         ax.scatter(*intersection.xy, color='yellow', label="Intersection", s = 100, zorder = 20)
    else:
        ax.plot([x_vertical, x_vertical], [0, 600], label="Vertical Line", linestyle='--', color='black', zorder = zorder)  # using a large y range for visualization
        ax.plot([x_extended_1, x_extended_2], [y_extended_1, y_extended_2], label="Extended Line", linestyle='--', color='black', zorder = zorder)
#         ax.scatter(*intersection.xy, color='yellow', label="Intersection", s = 100, zorder = 20)
    return intersection

def drawTrueTrajectory(condition, df, ax, zorder):
    subset = df[df['stimulus_idx'] == condition]
    trails = subset['exact_path_single']
    data_list = subset['exact_path_single'].iloc[0]
    x_coords = [item[0] for item in data_list]
    y_coords = [item[1] for item in data_list]
    ax.plot(x_coords, y_coords, '-', color='cyan', linewidth=2, zorder=zorder)
#     ax.scatter(x_coords, y_coords, c='cyan', s=4, zorder = zorder)
    
def drawSimulatedTrajectory(condition, trajectoryName, df, ax, zorder):
    subset = df[df['stimulus_idx'] == condition]
    trails = subset[trajectoryName]
    data_list = subset[trajectoryName].iloc[0]
    x_coords = [item[0] for item in data_list]
    y_coords = [item[1] for item in data_list]
    ax.scatter(x_coords, y_coords, c='red', s=2, zorder = zorder)
    
def drawHumanCentroid(condition, obstacle, humanCentroidJSON, ax, alpha, zorder):
    with open(humanCentroidJSON, 'r') as file:
        data = json.load(file)
    df_simulated = pd.json_normalize(data)
    
    trajectory = df_simulated.loc[(df_simulated['stimulus_idx'] == condition) & (df_simulated['obstacle_idx'] == obstacle), 'simulated_trial'].iloc[0]
    x_coords = [item[0] for item in trajectory]
    y_coords = [item[1] for item in trajectory]
    
#     ax.scatter(x_coords, y_coords, c='gray', s=4, alpha = alpha, zorder = zorder)
    
    # placement of the triangle
    x_triangle = df_simulated.loc[(df_simulated['stimulus_idx'] == condition) & (df_simulated['obstacle_idx'] == obstacle), 'obstacle_X'].iloc[0]
    y_triangle = df_simulated.loc[(df_simulated['stimulus_idx'] == condition) & (df_simulated['obstacle_idx'] == obstacle), 'obstacle_Y'].iloc[0]
#     print(x_triangle, y_triangle)
    return x_triangle, y_triangle

In [14]:
df_all = allData # to be improved

In [ ]:
intersections = {}
conditions = np.sort(df_all['obstacle_idx'].unique())

# Define a color map for your conditions
color_map = {1: 'red', 2: 'red', 3: '#757575', 4: '#757575', 5: '#0088DE', 6: '#0088DE'}

# Reshape positions into a 3x4 array and flatten by column
positions = np.sort(df_all['stimulus_idx'].unique())
positions = positions.reshape((3, 4)).T.flatten()

fig, axs = plt.subplots(4, 3, figsize=(28, 24)) # Creates a 4x3 grid of Axes objects

# Empty lists to hold all handles and labels
handles, labels = [], []

for index, position in enumerate(positions):
    ax = axs.flatten()[index]  # Select the current Axes object

    # Draw triangle and rectangle
    x_coord = df_all.loc[df_all['stimulus_idx'] == position, 'obstacle_groundTruth_x'].values[0]
    y_coord = df_all.loc[df_all['stimulus_idx'] == position, 'obstacle_groundTruth_y'].values[0]
    x_coord_ball = df_all.loc[df_all['stimulus_idx'] == position, 'ball_X'].values[0]
    y_coord_ball = 100
    print(x_coord, y_coord)
    ball = patches.Circle((x_coord_ball, y_coord_ball), radius = 30, color = '#ececd1')
    triangle = patches.RegularPolygon((x_coord,y_coord), orientation=np.pi, numVertices=3, radius=45, color='#5fa55a', fill=True)
    rect = patches.Rectangle((250,150),500,300,linewidth=1, edgecolor='lightgray',facecolor='lightgray')
    ax.add_patch(ball)
    ax.add_patch(rect)
    ax.add_patch(triangle)
    
    for i, condition in enumerate(conditions):
        subset = df_all[(df_all['stimulus_idx'] == position) & (df_all['obstacle_idx'] == condition)]
        ax.scatter(subset["triangle_final_x_flipback"], subset["triangle_final_y"], s=20, color=color_map[i+1], alpha = 0.6)
        
        # Plot initial positions
        initial_x = subset['obstacle_initial_x'].unique()
        initial_y = subset['obstacle_initial_y'].unique()
        ax.scatter(initial_x, initial_y, marker='x', color='white',linewidth=5, s=130, zorder=10)  
        scatter = ax.scatter(initial_x, initial_y, marker='x', color=color_map[i+1],linewidth=3, s=100, zorder=10)  

        # Add handles and labels to lists
        if len(handles) < len(conditions):
            handles.append(scatter)
            labels.append(f'Initial position: {int(condition)}')
    
    # plot the true trajectory
    drawTrueTrajectory(position, df_all, ax, 7)
    
    # plot the simulated trajectory of the participants' centroids (12 in total)
#     drawSimulatedTrajectory(position, 'simulated_trial', df_simulated, ax, 7)
#     drawSimulatedTrajectory(position, 'simulated_trial', df, ax, 7)
    
    # plot the projected path
    intersection = drawProjectedPath(position, df_all, ax, 7)
    intersections[position] = [intersection.x, intersection.y]
    
    ax.set_xlim(100, 900)
    ax.set_ylim(60, 540)
    ax.invert_yaxis() # invert the y-axis
    ax.set_xlabel("triangle_final_x")
    ax.set_title(f'Pos {int(position)}')
    
plt.tight_layout()  
# add color bar
# cax = fig.add_axes([1.05, 0.2, 0.02, 0.6])  # Adjust the position and size as needed
# cb = ColorbarBase(cax, cmap=hsv, orientation='vertical', norm=plt.Normalize(vmin=global_min, vmax=global_max))
# cb.set_label('Speed Value')

fig.legend(handles, labels, loc='upper right')
plt.show()

In [ ]:
intersections

In [16]:
# modify the intersections
# assume that participants always assume that the ball bounces off the midpoint of the triangle's sides
d = (30 + (45/2))/2*(3**0.5) # 45.46633369868302
fall_Direction = {1.0: 'left', 2.0: 'right', 3.0: 'right', 4.0: 'left', 
                 5.0: 'right', 6.0: 'right', 7.0: 'left', 8.0: 'left',
                 9.0: 'left', 10.0: 'right', 11.0: 'left', 12.0: 'right'}
intersections_modified = {}
for key in intersections:
    direction = fall_Direction[key]
    if direction == 'left':
        intersections_modified[key] = [intersections[key][0] + d, intersections[key][1]]
    else:
        intersections_modified[key] = [intersections[key][0] - d, intersections[key][1]]

In [ ]:
intersections_modified

In [17]:
intersections_modified_by_anchor = {}
for i in intersections_modified.keys():
    for j in range(1,7):
        new_key = (i,j)
        intersections_modified_by_anchor[new_key] = intersections_modified[i]

In [ ]:
intersections_modified_by_anchor # LP intersections by anchor, 72 intersections (condition, obstacle)

In [18]:
def plotComparison(x_human, y_human, x_model, y_model, std_dev_human_x, std_dev_human_y, label, std_dev_model_x = None, std_dev_model_y = None):
    # Creating the plots with error bars
    plt.figure(figsize=(10, 5))

    # Plot for X comparisons
    plt.subplot(1, 2, 1)
    plt.errorbar(x_model, x_human, xerr=std_dev_model_x, yerr=std_dev_human_x, fmt='o', color='blue', ecolor='lightgray', label='X Comparisons')
    xlim = plt.xlim()
    ylim = plt.ylim()
    line_range = [min(xlim[0], ylim[0]), max(xlim[1], ylim[1])]
    plt.plot(line_range, line_range, 'k--')
    plt.xlabel(f'{label} X', fontsize=16)
    plt.ylabel('Human X Responses', fontsize=16)
    plt.xlim(line_range)
    plt.ylim(line_range)
    plt.legend()

    # Plot for Y comparisons
    plt.subplot(1, 2, 2)
    plt.errorbar(y_model, y_human, xerr=std_dev_model_y, yerr=std_dev_human_y, fmt='o', color='green', ecolor='lightgray', label='Y Comparisons')
    xlim = plt.xlim()
    ylim = plt.ylim()
    line_range = [min(xlim[0], ylim[0]), max(xlim[1], ylim[1])]
    plt.plot(line_range, line_range, 'k--')
    plt.xlabel(f'{label} Y', fontsize=16)
    plt.ylabel('Human Y Responses', fontsize=16)
    plt.xlim(line_range)
    plt.ylim(line_range)
    plt.legend()

    plt.tight_layout()
#     plt.savefig(f'{label} vs. Human Predictions.pdf')
    plt.show()

In [ ]:
# Obtain data
x_human = df_all.groupby(['stimulus_idx','obstacle_idx'])['triangle_final_x_flipback'].mean().values # automatically sorted by 1, 2, 3, ...
y_human = df_all.groupby(['stimulus_idx','obstacle_idx'])['triangle_final_y'].mean().values
x_model = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_groundTruth_x'].first().values
y_model = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_groundTruth_y'].first().values

# Assuming some standard deviations for model and human predictions
std_dev_human_x = df_all.groupby(['stimulus_idx','obstacle_idx'])['triangle_final_x_flipback'].std().values
std_dev_human_y = df_all.groupby(['stimulus_idx','obstacle_idx'])['triangle_final_y'].std().values

# Plot comparison between human responses and ground-truth positions (separated by x/y)
plotComparison(x_human, y_human, x_model, y_model, std_dev_human_x, std_dev_human_y, 'Ground-truth')

#### 2D

In [19]:
def plotComparison_fd(human_fd, model_fd, std_dev_human_fd, label, std_dev_model_fd = None):
    # Creating the plots with error bars
    plt.figure(figsize=(5, 5))

    plt.errorbar(model_fd, human_fd, xerr=std_dev_model_fd, yerr=std_dev_human_fd, fmt='o', color='blue', ecolor='lightgray')

    xlim = plt.xlim()
    ylim = plt.ylim()
    line_range = [min(xlim[0], ylim[0]), max(xlim[1], ylim[1])]
    plt.plot(line_range, line_range, 'k--')
    plt.xlabel(f'{label} Evaluated by Fréchet Distance', fontsize=15)
    plt.ylabel('Human Evaluated by Fréchet Distance', fontsize=15)
    plt.xlim(line_range)
    plt.ylim(line_range)
    
    correlation_fd, p_value_fd = pearsonr(human_fd, model_fd)
#     print(model_fd)
    print("p-value:", p_value_fd)
    print("Pearson Correlation:", correlation_fd)
    
    plt.title(f'Pearson Correlation: {correlation_fd:.3f}', fontsize=15)
    plt.savefig(f'Figs_Update_3/{label} vs. Human Predictions_FD.pdf')
    plt.show()
    return correlation_fd

## 2D comparison

In [20]:
def calculate_total_likelihood(cov_params, all_data, all_means):
    """
    Calculate the total log likelihood for all conditions.
    """
    # Reconstruct the covariance matrix from the parameters
    L = np.array([[cov_params[0], 0], [cov_params[1], cov_params[2]]])
    cov = L @ L.T

    total_likelihood = 0
#     count = 0
    for data, mean in zip(all_data, all_means):
        total_likelihood -= np.sum(multivariate_normal.logpdf(data, mean=mean, cov=cov)) # use log(ab)=log(a)+log(b), so just summing loglikelihoods instead of logsumexp
    return total_likelihood

def find_single_best_covariance(all_data, all_means, method):
    # Initial guess for covariance matrix parameters
    initial_cov_params = [10, 0, 10]
    
    # Ensuring that the diagonal elements are positive & covariance = 0
    bounds = Bounds([0, 0, 0], [np.inf, 0, np.inf])

    # Run optimization to find the best covariance matrix
    result = minimize(calculate_total_likelihood, initial_cov_params, args=(all_data, all_means), bounds=bounds, method = method)

    # Reconstruct the optimized covariance matrix
#     print(result)
    L_optimized = np.array([[result.x[0], 0], [result.x[1], result.x[2]]])
    optimized_cov = L_optimized @ L_optimized.T
    
    # result.fun is the minimized negative log likelihood, so we take the negative to get the log likelihood
    return optimized_cov, -result.fun

# Example usage
# Aggregate data and means for all conditions
# Assuming you have a list of data arrays and corresponding means for each condition
# Example:
# all_data = [data_condition1, data_condition2, ..., data_conditionN]
# all_means = [mean_condition1, mean_condition2, ..., mean_conditionN]

# optimized_covariance = find_best_covariance(all_data, all_means)
# print(optimized_covariance)

#### if single covariance matrix that maximizes the sum of the maximum likelihoods calculated per condition

In [21]:
positions = np.sort(df_all['stimulus_idx'].unique())
anchors = np.sort(df_all['obstacle_idx'].unique())

def getBICFromPrediction(x_model, y_model, df_all, k):
    positions = np.sort(df_all['stimulus_idx'].unique())
    anchors = np.sort(df_all['obstacle_idx'].unique())

    # Aggregate data from all conditions, all anchors
    all_data = np.array([df_all[(df_all['stimulus_idx'] == pos) & (df_all['obstacle_idx'] == anc)][['triangle_final_x_flipback', 'triangle_final_y']].values for pos in positions for anc in anchors])
    # print(len(all_data))

    # Mean models for all conditions
    all_mean_models = np.array([x_model, y_model]).T
    # print(all_mean_models)

    # Find a single best-fit covariance matrix for all conditions
    optimized_covariance, mle_all = find_single_best_covariance(all_data, all_mean_models, 'Powell')
    # optimized_covariance, mle_all = find_single_best_covariance(all_data, all_mean_models, None)
    
    # Ensure mle_all is a scalar
    assert np.isscalar(mle_all), "mle_all should be a scalar value."

    print(f"Best sum log-likelihood: {mle_all}")
    print(f'Best covariance: {optimized_covariance}')
    print(f'BIC: {getBIC(k, 1440, mle_all)}')
    return mle_all

In [22]:
# a function that calculates the fd between ground-truth trajectory and the simulated trajectory of model prediction (stored in jsonFilePath)
def compareFD_model_human(jsonFilePath, df_all, label):
    with open(jsonFilePath, 'r') as file:
        data = json.load(file)

    # Convert JSON data to DataFrame and sort it
    df_simulated = pd.json_normalize(data)
    df_simulated = df_simulated.sort_values(by=['stimulus_idx', 'obstacle_idx']).reset_index(drop=True)
    
    # Obtain and sort the ground-truth paths
    truePaths = df_all.groupby(['stimulus_idx','obstacle_idx'])['exact_path_single'].agg('first').reset_index()
    truePaths = truePaths.sort_values(by=['stimulus_idx', 'obstacle_idx']).reset_index(drop=True)
    
    # Merge simulated data with true paths correctly using an inner join to ensure alignment
    df_merge_simulation_truePaths = pd.merge(df_simulated, truePaths, on=['stimulus_idx', 'obstacle_idx'], how='inner')
    df_merge_simulation_truePaths = df_merge_simulation_truePaths.rename(columns={"simulated_trial": "simulated_path"})
    
    evals_fd = df_merge_simulation_truePaths.apply(evaluateResponseByFD, axis=1)
    evals_df = evals_fd.apply(pd.Series)
    evals_df.columns = ['fd1_enter', 'fd2_exit']
    evals_df['fd_combined'] = evals_df['fd1_enter'] + evals_df['fd2_exit']

    df_merge_simulation_truePaths = pd.concat([df_merge_simulation_truePaths, evals_df], axis=1)
    
    # Calculate the means and standard deviations for human and model FD
    human_fd = df_all.groupby(['stimulus_idx', 'obstacle_idx'])['fd_combined'].mean().sort_index().values
    model_fd = df_merge_simulation_truePaths['fd_combined'].values

    std_dev_human_fd = df_all.groupby(['stimulus_idx', 'obstacle_idx'])['fd_combined'].std().sort_index().values

    corr_fd = plotComparison_fd(human_fd, model_fd, std_dev_human_fd, label)

    return corr_fd 

In [24]:
def storeXYPrediction(x_model, y_model, modelName):
    if len(x_model) != 72 or len(y_model) != 72:
        print('CHECK Model Predictions. Length != 72.')
        if len(x_model) == 12 and len(y_model) == 12:
            x_model = [x_sol for x_sol in x_model for _ in range(6)]
            y_model = [y_sol for y_sol in y_model for _ in range(6)]
    model_dir = Path(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}')
    if os.path.exists(f'{model_dir}/x_coords_{modelName}.pkl') and os.path.exists(f'{model_dir}/y_coords_{modelName}.pkl'):
        print('Model Predictions Stored Already!')
    else:
        
        model_dir.mkdir(parents=True, exist_ok=True)
        with open(f'{model_dir}/x_coords_{modelName}.pkl', 'wb') as f:
            pickle.dump(x_model, f)
        with open(f'{model_dir}/y_coords_{modelName}.pkl', 'wb') as f:
            pickle.dump(y_model, f)

## Code for visualizing the simulated path

### Get 72 human centroids & Run simulations & analysis

In [25]:
# get 72 human centroids (from participant)
x_human = []
y_human = []

for stimulus_idx in positions:
    for obstacle_idx in anchors:
        subset = df_all[(df_all['stimulus_idx'] == stimulus_idx) & (df_all['obstacle_idx'] == obstacle_idx)]
        x_mean = subset['triangle_final_x_flipback'].mean()
        y_mean = subset['triangle_final_y'].mean()
#         print(x_mean, y_mean)
        x_human.append(x_mean)
        y_human.append(y_mean)

# storeXYPrediction(x_human, y_human, 'Human')

In [26]:
# runScript_72Simulations('runSimulation/full72Trajectories/run72Simulations.py', 'Human')

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/Human/all72_results.json', df_all)

In [ ]:
compareFD_model_human(f'runSimulation/full72Trajectories/Human/all72_results.json', df_all, 'Human_Centroid')
# if we compare the trajectories of the human centroids with the average fd achieved by the participants in each condition
# the correlation is pretty low
# this suggests that fitting the centroid (which explains placement positions) vs. fitting the response quality are two different goals

In [27]:
def drawInitialSetUp(ax, df_all, stimulus_idx, obstacle_idx):
    # Draw background rectangle as the screen
    screen = patches.Rectangle((250,150), 500, 300, linewidth=1, edgecolor=screen_color, facecolor=screen_color)
    ax.add_patch(screen)

    # Add the ball using the specific x-coordinate from the data
    ball = patches.Circle((ball_Xs[stimulus_idx-1], 100), radius=30, color=ball_color)
    ax.add_patch(ball)

    # Add the ground truth triangle position
    x_coord = df_all.loc[df_all['stimulus_idx'] == stimulus_idx, 'obstacle_groundTruth_x'].values[0]
    y_coord = df_all.loc[df_all['stimulus_idx'] == stimulus_idx, 'obstacle_groundTruth_y'].values[0]
    triangle = patches.RegularPolygon((x_coord, y_coord), numVertices=3, radius=45, orientation=np.pi, color=ground_truth_color, fill=True)
    ax.add_patch(triangle)

    # Add the anchor position # orange
    subset = df_all[(df_all['stimulus_idx'] == stimulus_idx) & (df_all['obstacle_idx'] == obstacle_idx)]
    initial_x = subset['obstacle_initial_x'].unique()
    initial_y = subset['obstacle_initial_y'].unique()
    ax.scatter(initial_x, initial_y, marker='x', color=anchor_color,linewidth=3, s=60,zorder=8) 
    
    # Plot true trajectory # cyan
    drawTrueTrajectory(stimulus_idx, df_all, ax, 5)
    
    # Plot human centroid trajectory & obstacle # dark gray
    human_X, human_Y = drawHumanCentroid(stimulus_idx, obstacle_idx, 'runSimulation/full72Trajectories/Human/all72_results.json', ax, 0.5, 4)
    human_avg_response = patches.RegularPolygon((human_X, human_Y), numVertices=3, radius=45, orientation=np.pi, color=human_centroid_color, fill=True, alpha=0.3)
    ax.add_patch(human_avg_response)

    # Plot Participants Solutions
    for path in subset['simulated_path']:
        path = np.array(path)  # Make sure it's a numpy array for indexing
        ax.plot(path[:, 0], path[:, 1], '-', color=participant_color, linewidth=2, zorder=5, alpha=0.5)

    return subset

In [28]:
# new function with legend
def visualize72Trials(jsonFilePath, df_all):
    # Load JSON data, containing simulated trajectories from model prediction
    with open(jsonFilePath, 'r') as file:
        data = json.load(file)

    # Convert data to DataFrame
    df = pd.json_normalize(data)

    # Create a 12x6 grid of plots with shared axes and constrained layout
    fig, axes = plt.subplots(nrows=12, ncols=6, figsize=(30, 40), sharex=True, sharey=True, constrained_layout=True)

    # Loop through each row in the dataframe
    for idx, row in df.iterrows():
        stimulus_idx = row['stimulus_idx'] - 1  # 0-based index for matplotlib
        obstacle_idx = row['obstacle_idx'] - 1  # 0-based index

        ax = axes[stimulus_idx, obstacle_idx]
        ax.set_aspect('equal')
        
        subset = drawInitialSetUp(ax, df_all, row['stimulus_idx'], row['obstacle_idx'])

        # Add the model predicted obstacle position
        obstacle = patches.RegularPolygon((row['obstacle_X'], row['obstacle_Y']), numVertices=3, radius=45, orientation=np.pi, color=model_prediction_color, fill=True, alpha=0.5)
        ax.add_patch(obstacle)

        # Plot trajectory
        x_coords = [point[0] for point in row['simulated_trial']]
        y_coords = [point[1] for point in row['simulated_trial']]
        ax.plot(x_coords, y_coords, '-', linewidth=1.5, color='darkblue', label='Trajectory', zorder=6)

        ax.set_xlim(100, 900)
        ax.set_ylim(60, 540)
        ax.invert_yaxis()
        ax.set_title(f'Stimulus {row["stimulus_idx"]}, Obstacle {row["obstacle_idx"]}')

    # Add a single global legend
    handles = [
        patches.Patch(color=ground_truth_color, label='Ground Truth'),
        patches.Patch(color=model_prediction_color, label='Model Prediction'),
        patches.Patch(color=participant_color, label='Participant Trajectories'),
        patches.Patch(color=human_centroid_color, label='Human Centroid'),
#         patches.Patch(color=ball_color, label='Ball'),
#         patches.Patch(color=screen_color, label='Screen'),
        patches.Patch(color=anchor_color, label='Anchor Position')
    ]
    fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.01), ncol=5, fontsize='16')

    fig.supxlabel('X Coordinate', fontsize=16)
    fig.supylabel('Y Coordinate', fontsize=16)
    plt.show()

# This code assumes you have the jsonFilePath and df_all prepared
# visualize72Trials('path_to_your_data.json', df_all_dataframe)

## Initialization

### Blue Region

X (based on falling direction)
(x_ball - l, x_ball)  OR
(x_ball, x_ball + l)

l = r_ball + \sqrt{3}/2*r_obstacle

Y: (150, 450) (occluder top/bottom)

In [29]:
# width of the blue region
l = ball_radius + 3**0.5/2*obstacle_radius

# x_range for stimulus_idx in positions for obstacle_idx in anchors
x_blue_range = {}

for stimulus_idx in positions:
    for obstacle_idx in anchors:
        direction = fall_Direction[stimulus_idx]
        x_ball = intersections[stimulus_idx][0]
        # To make sure the edges of the blue region are exclusive (also to ensure valid collision), 
        # a tiny margin of 1 is applied. E.g. (150, 350) -> [149, 349].
        margin = 1
        # if the ball is falling towards the left
        # x_min = x_ball + adjustment
        # x_max = x_ball + l - adjustment, (otherwise, when x = x_ball + l, no collision would happen)
        if direction == 'left':
            range_vals = (x_ball + margin, x_ball + l - margin)
        else:
            range_vals = (x_ball - l + margin, x_ball - margin)
            
        x_blue_range[stimulus_idx, obstacle_idx] = range_vals

In [ ]:
x_blue_range

## Random

In [31]:
def runScript_72Simulations(script_path, modelName):
    if not os.path.exists(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json'):
        try:
            print(f"Running script for: {modelName}")
            result = subprocess.run(
                ['python', script_path, modelName],
                text=True,  # Ensures outputs are returned as strings
                capture_output=True  # Captures output and errors
            )
            print(f"Output for {modelName}:\n{result.stdout}")
            if result.stderr:
                print(f"Errors for {modelName}:\n{result.stderr}")
            print(f"Completed script for: {modelName}")

        except subprocess.CalledProcessError as e:
            print(f"An error occurred while running script for {modelName}: {e}")
    
    else:
        print(f"Results for model {modelName} already exist.")

In [ ]:
np.random.seed(42)

# y_model is randomly drawn between 151, 449
# x_model is randomly drawn between x_blue_range

MLL_list = []
corr_list = []
for i in range(30):
    x_model = [np.random.uniform(x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for stimulus_idx in positions for obstacle_idx in anchors]
    y_model = np.random.uniform(150 + margin, 450 - margin, 72)
    modelName = f'random_{i}'
    # store x,y coords
    storeXYPrediction(x_model, y_model, modelName)
    
    # get maximum likelihoods
    MLL = getBICFromPrediction(x_model, y_model, df_all, 2)
    MLL_list.append(MLL)
    
    # run 72 simulations, if not yet run
#     if not os.path.exists(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json'):
    runScript_72Simulations(f'runSimulation/full72Trajectories/{modeling_choice}/run72Simulations.py', modelName)
    
    # calculate correlations
    corr = compareFD_model_human(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json', df_all, 'Init$_{Random}$')
    corr_list.append(corr)

from scipy.special import logsumexp

# logsumexp considers the sum of exponentiated log-likelihoods, 
# which corresponds to combining probabilities before returning to log-space
adjusted_logsumexp = logsumexp(MLL_list) - np.log(len(MLL_list))
getBIC(2, 1440, adjusted_logsumexp) # 31531.8525 # corrected Sep_29: 31220.487927639057

In [ ]:
getBIC(2, 1440, adjusted_logsumexp) # 31220.487927639057

In [ ]:
# get the average mll across the 30 samples
adjusted_logsumexp # -15602.971565426958

In [ ]:
np.mean(corr_list) # 0.2816672581685109

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/{modeling_choice}/random_14/all72_results.json',df_all) # random_9,14

## Linear Projection

### # issue: when modeling x at the same time, the x coordinate determined by LP was x_ball. However, set x_model to be x_ball might result in invalid collision (50/50 falling into either side)

## LP_approach 1: make x random? Since LP could only fix y

In [ ]:
np.random.seed(42)

# y_model is set to be LP positions
# x_model is randomly drawn between x_blue_range

MLL_list_LP = []
corr_list_LP = []
for i in range(30):
    x_model = [np.random.uniform(x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for stimulus_idx in positions for obstacle_idx in anchors]
    y_model = y_model = [intersections_modified_by_anchor[(stimulus_idx,obstacle_idx)][1] for stimulus_idx in positions for obstacle_idx in anchors]
    modelName = f'LP_xRandom_{i}'
    # store x,y coords
    storeXYPrediction(x_model, y_model, modelName)
    
    # get maximum likelihoods
    MLL = getBICFromPrediction(x_model, y_model, df_all, 2)
    MLL_list_LP.append(MLL)
    
    # run 72 simulations, if not yet run
    runScript_72Simulations(f'runSimulation/full72Trajectories/{modeling_choice}/run72Simulations.py', modelName)
    
    # calculate correlations
    corr = compareFD_model_human(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json', df_all, 'Init$_{LP_xRandom}$')
    corr_list_LP.append(corr)

getBIC(2, 1440, np.mean(MLL_list_LP)) # 32774.084016321496

In [ ]:
print(np.mean(corr_list_LP)) # 0.5027082569370811
print(np.mean(MLL_list_LP)) # -16379.769609768176

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/{modeling_choice}/LP_xRandom_29/all72_results.json',df_all)

## approach 2 : fix x as the previous approach, assume collision midpoint, adjusted by l_0

In [ ]:
modelName = 'Linear_Projection'
x_model = [intersections_modified_by_anchor[(stimulus_idx,obstacle_idx)][0] for stimulus_idx in positions for obstacle_idx in anchors] #(1,1)(1,2)(1,3)...(1,6)(2,1)
# x_model = [np.random.uniform(x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for stimulus_idx in positions for obstacle_idx in anchors]
y_model = [intersections_modified_by_anchor[(stimulus_idx,obstacle_idx)][1] for stimulus_idx in positions for obstacle_idx in anchors]

storeXYPrediction(x_model, y_model, modelName)

getBICFromPrediction(x_model, y_model, df_all, 2)

# Best sum log-likelihood: -16258.582836180341
# Best covariance: [[  892.56781217     0.        ]
#  [    0.         24623.67209481]]
# BIC: 32531.710469145823

runScript_72Simulations(f'runSimulation/full72Trajectories/{modeling_choice}/run72Simulations.py', modelName)

In [ ]:
x_model

In [ ]:
compareFD_model_human(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json', df_all, 'Init$_{LP}$')
# p-value: 4.664381787366499e-07
# Pearson Correlation: 0.5532128025051875

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json',df_all)

## approach 3: The point in the blue region that is nearest to the intersection.

In [ ]:
modelName = 'LP_near'
x_model = [nearest_value_in_range(intersections[stimulus_idx][0],x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for stimulus_idx in positions for obstacle_idx in anchors] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_model = [nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for stimulus_idx in positions for obstacle_idx in anchors]

storeXYPrediction(x_model, y_model, modelName)

getBICFromPrediction(x_model, y_model, df_all, 2)

# Best sum log-likelihood: -16180.560490561125
# Best covariance: [[ 1528.13625303     0.        ]
#  [    0.         12906.43108133]]
# BIC: 32375.66577790739


runScript_72Simulations(f'runSimulation/full72Trajectories/{modeling_choice}/run72Simulations.py', modelName)

In [ ]:
compareFD_model_human(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json', df_all, 'Init$_{LP_{near}}$')
# p-value: 3.678311654244363e-05
# Pearson Correlation: 0.4661510063296203

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json',df_all)

## Anchor

## even anchor could help with fixing some of the x_position, it sometimes will still return x_ball. However, can not return the exact ball location, since will cause invalid collision

Solution: set a length 1 margin for the blue region, so that a valid collision will always happen.

In [41]:
def nearest_value_in_range(x, range_start, range_end):
    if x < range_start:
        return range_start
    elif x > range_end:
        return range_end
    else:
        return x
    

In [ ]:
positions = np.sort(df_all['stimulus_idx'].unique())
anchors = np.sort(df_all['obstacle_idx'].unique())
anchor_Xs = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_initial_x'].first()

modelName = 'Anchor'
x_model = [nearest_value_in_range(anchor_Xs[stimulus_idx][obstacle_idx],x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for stimulus_idx in positions for obstacle_idx in anchors] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_model = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_initial_y'].first().values

storeXYPrediction(x_model, y_model, modelName)

getBICFromPrediction(x_model, y_model, df_all, 2)

# Best sum log-likelihood: -15555.833691892942
# Best covariance: [[1672.2231159     0.        ]
#  [   0.         4952.72557391]]
# BIC: 31126.212180571023
    
runScript_72Simulations(f'runSimulation/full72Trajectories/{modeling_choice}/run72Simulations.py', modelName)

In [ ]:
compareFD_model_human(f'runSimulation/full72Trajectories/{modeling_choice}/Anchor/all72_results.json', df_all, 'Init$_{Anchor}$')
# p-value: 1.3602436172697692e-06
# Pearson Correlation: 0.5340025642116222

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/{modeling_choice}/Anchor/all72_results.json',df_all)

## Midline

In [ ]:
modelName = 'Midline'
x_model = [nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for stimulus_idx in positions for obstacle_idx in anchors] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_model = [nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for stimulus_idx in positions for obstacle_idx in anchors]

storeXYPrediction(x_model, y_model, modelName)

getBICFromPrediction(x_model, y_model, df_all, 2)
# Best sum log-likelihood: -15098.924627238646
# Best covariance: [[1714.38214377    0.        ]
#  [   0.         2561.01649376]]
# BIC: 30212.394051262432
runScript_72Simulations(f'runSimulation/full72Trajectories/{modeling_choice}/run72Simulations.py', modelName)

In [ ]:
compareFD_model_human(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json', df_all, 'Init$_{Mid}$')

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json',df_all)

# With Adjustment

# (2) Linear projection followed by refinement with a physics engine. 

The basic idea is that you initialize the triangle location at the linear projection hypothesis, then run a simulation to see where the ball goes. Depending on whether it's above or below the exit point, adjust the triangle up or down along the linear vertical path from the initial ball position. The parameters here would be the size of the increment and the stopping criterion (number of increments or some error threshold). It would be easier (as a start) to use a deterministic model (no noise, as in Kevin's model).

#### Do you know if people take longer to make a response for colliders farther from the midline?

In [ ]:
# average response time per trial
df_all['response_time'].mean() # 5233.517361107675 around 5s

In [ ]:
df_all['sol_distance_from_midline'] = abs(df_all['triangle_final_y'] - 300)
df_all['gt_distance_from_midline'] = abs(df_all['obstacle_groundTruth_y'] - 300)

sns.set_theme(style="darkgrid")
# sns.scatterplot(x='sol_distance_from_midline', y='response_time', data=df_all)
sns.regplot(x='sol_distance_from_midline', y='response_time', data=df_all, line_kws={"color":"#7F00FF"}, scatter_kws={"s":5, "color":"m"})
plt.xlabel('sol_distance_from_midline', fontsize=18)
plt.ylabel('Response Time', fontsize=18)

# Use tight_layout to automatically adjust subplot parameters to give specified padding
plt.tight_layout()
# plt.title('Scatter Plot of Distance_From_Mid vs Response Time')
plt.savefig('Sol_Distance_From_Mid vs RT.pdf')
plt.show()

In [ ]:
# Calculating regression
from scipy.stats import linregress
regression_result = linregress(df_all['sol_distance_from_midline'], df_all['response_time'])
print(f"Slope: {regression_result.slope}")
print(f"Intercept: {regression_result.intercept}")
print(f"R-squared: {regression_result.rvalue**2}")

In [ ]:
regression_result

In [ ]:
# Adding a constant to the model
X = sm.add_constant(df_all['sol_distance_from_midline'])  # Independent variable
y = df_all['response_time']  # Dependent variable

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the model
print(model.summary())

In [ ]:
# sns.scatterplot(x='gt_distance_from_midline', y='response_time', data=df_all)
sns.regplot(x='gt_distance_from_midline', y='response_time', data=df_all, line_kws={"color":"#7F00FF"}, scatter_kws={"s":5, "color":"m"})
plt.xlabel('gt_distance_from_midline', fontsize=18)
plt.ylabel('Response Time', fontsize=18)
# plt.title('Scatter Plot of Distance_From_Mid vs Response Time')
# Use tight_layout to automatically adjust subplot parameters to give specified padding
plt.tight_layout()
plt.savefig('GT_Distance_From_Mid vs RT.pdf')
plt.show()

In [ ]:
regression_result_gt = linregress(df_all['gt_distance_from_midline'], df_all['response_time'])
# print(f"Slope: {regression_result_gt.slope}")
# print(f"Intercept: {regression_result_gt.intercept}")
# print(f"R-squared: {regression_result_gt.rvalue**2}")
print(regression_result_gt)

## Determine the Exit Point

In [ ]:
exitPoints_two = allData.groupby('stimulus_idx')['projected_line_1'].first().values
exitPoints_single = [points[0] for points in exitPoints_two] # pick one point from the line
print(len(exitPoints_single))
# x_projs = x_model
# print(x_projs)
# y_projs = y_model
# print(y_projs)
ball_Xs = [322.3, 322.3, 322.3, 322.3, 500 , 500 , 500 , 500 , 604.4, 604.4, 604.4, 604.4]

In [53]:
def getClosestPoint(point, point_list):
    x, y = point
    # Use min with a key function that calculates the absolute difference between x and x*.
    # this criteria is when x the same, get the position of y
    closest_point = min(point_list, key=lambda p: abs(p[0] - x))
    return closest_point

In [ ]:
x_gt_model = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_groundTruth_x'].first().values
y_gt_model = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_groundTruth_y'].first().values
print(x_gt_model[0],y_gt_model[0])

In [140]:
# very similar to getBICFromPrediction, instead of return the BIC, only return the mle_all
def find_best_cov_model2(x_model, y_model, df_all):
    positions = np.sort(df_all['stimulus_idx'].unique())
    anchors = np.sort(df_all['obstacle_idx'].unique())
    
    # Aggregate data from all conditions
    all_data = np.array([df_all[(df_all['stimulus_idx'] == pos) & (df_all['obstacle_idx'] == anc)][['triangle_final_x_flipback', 'triangle_final_y']].values for pos in positions for anc in anchors])
#     print(len(all_data),len(all_data[0]))
    
    # Mean models for all conditions
    all_mean_models = np.array([x_model, y_model]).T
    print(len(all_mean_models),len(all_mean_models))

    # Find a single best-fit covariance matrix for all conditions
    optimized_covariance, mle_all = find_single_best_covariance(all_data, all_mean_models, 'Powell')
    
    return optimized_covariance, mle_all

## Converting the above code into OOP 

## Adjustment Method 1

In [141]:
# adjustment for a single set of parameters given a condition (stimulus_idx, obstacle_idx)
class SimulationScenario:
    def __init__(self, df_all, stimulus_idx, obstacle_idx, x_init, y_init, ball_Xs, x_blue_range, exitPoints_single, initChoice, stopSignal, maxSteps, debug = False):
        self.stimulus_idx = stimulus_idx # stimulus_idx, 1-12, int
        self.obstacle_idx = obstacle_idx # anchor_idx, 1-6, int
        self.x_init = x_init # initial x coords of the obstacle, 2-d list, [stimulus_idx, obstacle_idx], shape: (12,6)
        self.y_init = y_init # initial y coords of the obstacle, 2-d list, [stimulus_idx, obstacle_idx], shape: (12,6)
        self.ball_X = ball_Xs[stimulus_idx - 1] # initial ball position (to be fed into the js script), len(ball_Xs)=12
        self.exit_point = exitPoints_single[stimulus_idx - 1] # actual exit point of the ball, len(exitPoints_single)=12
        self.initChoice = initChoice # choice of initialization, ['LP', 'Anchor', 'Mid'], str
        self.stopSignal = stopSignal # choice of stopping criteria, ['exitPoint', 'pathSim'], str
        self.maxSteps = maxSteps # number of max adjustment, [np.inf, 0, 1, 2, 5, 10, 15, 20, 25, 30, 50]
        self.x_blue_range = x_blue_range

        # Filter dataset for the specific stimulus and obstacle index
        self.subset = df_all[(df_all['stimulus_idx'] == stimulus_idx) & (df_all['obstacle_idx'] == obstacle_idx)]
        self.exactPath = self.subset['exact_path_single'].iloc[0]  # Extract the exact path of the first row (the same for all rows in the subset) for comparison
        self.exactPath_entry, self.exactPath_exit = pickPointsOutScreen(self.exactPath, top_y, bottom_y, left_x, right_x, ball_radius)
        
        self.x_sol = None
        self.y_sol = None
        self.n_steps = 0 
        
        # track the adjustment history
        self.sol_history = []
        self.sol_fd_history = []
        self.simulated_path_history = []

        # printing & debugging
        self.debug = debug
        
    def set_initial_solution(self):
        """
        Set the initial x and y solution based on the initialization choice.
        """
        self.x_sol = self.x_init[self.stimulus_idx - 1][self.obstacle_idx - 1]
        # 'LP' (Linear Projection)/'Mid' initialization
        if self.initChoice in ['LP', 'Mid']:
            self.y_sol = self.y_init[self.stimulus_idx - 1][self.obstacle_idx - 1]
        # 'Anchor' initialization <-- old code, corrected for improvements
        elif self.initChoice == 'Anchor':
            self.y_sol = self.y_init[self.stimulus_idx][self.obstacle_idx]
#         Heuristic adjustment: if initial y position is outside the screen, bring it within the screen bounds
        if self.y_sol <= 150: self.y_sol = 150 # might be also corrected by setting the blue region
        

    def create_directory(self):
#         self.directory = os.path.dirname(f'runSimulation/Model2_by72Trials/{self.stimulus_idx}-{self.obstacle_idx}/')
        self.directory = os.path.dirname(f'runSimulation/Model2_by72Trials/Simulation Results-All/')
        if not os.path.exists(self.directory):
            os.makedirs(self.directory, exist_ok=True)
        
        # Define file path for simulation results
        self.file_path = f'{self.directory}/simulation-results-{self.ball_X}-{int(self.x_sol)}-{int(self.y_sol)}.json'

    def run_simulation(self):
        self.create_directory()
        if not os.path.exists(self.file_path):
            subprocess.run([
                "node", "runSimulation/Model2_by72Trials/simulateFunc.js",
                str(self.stimulus_idx), str(self.obstacle_idx), str(self.ball_X), str(self.x_sol), str(self.y_sol)
            ], check=True)
    
        # Load simulation results
#         print(self.file_path)
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        self.simulation_results = pd.json_normalize(data)
        self.simulated_path = self.simulation_results['simulated_trial'].iloc[0]
#         print(self.simulated_path)
        
    def get_closest_point(self, point_list):
        """
        Find the point in point_list that is horizontally closest to the actual exit point.
        """
        # Use min with a key function that calculates the horizontal distance
        closest_point = min(point_list, key=lambda p: (p[0] - self.exit_point[0])**2)
        return closest_point
    
    def eval_by_FD(self):
        # Calculate Frechet distances between simulated and exact paths for points out of screen
#         l1_p, l2_p = pickPointsOutScreen(self.simulated_path, top_y, bottom_y, left_x, right_x, ball_radius)
#         l1_g, l2_g = pickPointsOutScreen(self.exactPath, top_y, bottom_y, left_x, right_x, ball_radius)
        self.simulatedPath_entry, self.simulatedPath_exit = pickPointsOutScreen(self.simulated_path, top_y, bottom_y, left_x, right_x, ball_radius)
        if len(self.simulatedPath_exit) != 0 and len(self.simulatedPath_entry) != 0:
            self.sol_fd1 = frechet_dist(self.simulatedPath_entry, self.exactPath_entry)
            self.sol_fd2 = frechet_dist(self.simulatedPath_exit, self.exactPath_exit)
            if self.debug:
                print('fd_1, fd_2: ')
                print(self.sol_fd1, self.sol_fd2)
            self.sol_fd_history.append(self.sol_fd1+self.sol_fd2) # every time fd is evaluated, the result is appended, so fd_history length is not the actual adjustment 
        else: 
            print(self.file_path)
            print(self.stimulus_idx, self.obstacle_idx)
            raise Exception(f"Please Check the Source File {self.file_path}")
        
    def out_of_valid_region(self):
        # new bound
        if self.x_sol < self.x_blue_range[(self.stimulus_idx,self.obstacle_idx)][0] or self.x_sol > self.x_blue_range[(self.stimulus_idx,self.obstacle_idx)][1]:
            if self.debug:
                print(self.x_sol)
                print(f'Bound: {self.x_blue_range[(self.stimulus_idx,self.obstacle_idx)][0], self.x_blue_range[(self.stimulus_idx,self.obstacle_idx)][1]}')
                print('Out of Bound, Stop Adjust')
            return True
        if self.y_sol < 150 + margin or self.y_sol > 450 - margin:
            if self.debug:
                print(self.y_sol)
                print('Out of Bound, Stop Adjust')
            return True
        return False

        
    def stop_adjust(self, epsilon):
        """
        Determine if the adjustment process should stop based on the stopping criteria.
        """
        
        # Terminate if the solution has been evaluated before
        # if any(np.isclose(self.x_sol, x) and np.isclose(self.y_sol, y) for x, y in self.sol_history):
        # # if [self.x_sol, self.y_sol] in self.sol_history:
        #     if self.debug:
        #         print('In adjust history, Stop Adjust')
        #         print([self.x_sol, self.y_sol], self.sol_history)
        #     return True

        # out of bounds
        if self.out_of_valid_region():
            return True
        
        # Stopping criteria
        if self.stopSignal == 'exitPoint':
            # Calculate delta_y within the method and removed delta_y as a parameter
            x_e, y_e = self.get_closest_point(self.simulated_path)
            delta_y = self.exit_point[1] - y_e
            return abs(delta_y) < epsilon
        
        elif self.stopSignal == 'pathSim':
            return self.sol_fd1 + self.sol_fd2 < epsilon  # Epsilon is a threshold for path similarity
        
        return False
    
    
    # New Adjustment function (method 1), adjust in both x and y, loop through 8 directions, choose the best
    def adjust_obstacle(self, d, epsilon):
        """
        Adjust the obstacle's position to align the simulated exit point with the actual exit point.

        Parameters:
        - d: The magnitude of each adjustment step.
        - epsilon: The stopping criteria tolerance for vertical distance to the exit point/exit path similarity
        """
        # Set initial solution based on the chosen initialization method
        self.set_initial_solution()

        # Run simulation for the initialization
        self.run_simulation() # get self.simulated_path
        self.eval_by_FD() # append fd for the first time, get self.sol_fd1, self_sol_fd2

        # initialize best result
        opt_distance = (self.sol_fd1 + self.sol_fd2) if self.stopSignal == 'pathSim' else abs(self.exit_point[1] - self.get_closest_point(self.simulated_path)[1])

        # Main loop for the adjustment process

        # if no adjustment needed, then record the results and return
        if self.stop_adjust(epsilon):
#             print('Adjustment Stops')
            self.sol_history.append([self.x_sol, self.y_sol])
            self.simulated_path_history.append(self.simulated_path)
        
        # otherwise, start adjustment
        else:
#             print('Adjustment Continues')
            # add current solution to the history
            self.sol_history.append([self.x_sol, self.y_sol])
            self.simulated_path_history.append(self.simulated_path)
            
            # start adjustment loop
            d_diag = d/2*(2**0.5) # adjust by d_diag in x/y direction
            while self.n_steps < self.maxSteps:
                if self.debug:
                    if self.n_steps % 10 == 1:
                        print(f'steps: {self.n_steps}')
                        print(x_blue_range[(self.stimulus_idx,self.obstacle_idx)])
                        print(self.x_sol, self.y_sol)
                        print(f'Checking Stopping Criteria, best outcome: {opt_distance}')
#                 if np.random.random() < 0.005: 
#                     print(':) Adjusting')
#                     print(self.n_steps)
#                     print(self.x_sol, self.y_sol)
                # Define directions for adjustment: (dx, dy)
                directions = [
                    (0, -d),  # Up
                    (0, d),  # Down
                    (-d, 0),  # Left
                    (d, 0),  # Right
                    (d_diag, -d_diag),  # Right-Up
                    (d_diag, d_diag),  # Right-Down
                    (-d_diag, -d_diag),  # Left-Up
                    (-d_diag, d_diag)  # Left-Down
                ]

                # Evaluate all directions and select the best one
                results = []
                if self.debug:
                    print('Looping through directions')
                    print(f'Old Result: {self.x_sol, self.y_sol}')

                # Store the original solution to revert after trying each adjustment
                original_x, original_y = self.x_sol, self.y_sol
                
                for dx, dy in directions:
                    # Adjust x and y positions based on the direction
                    new_x_sol = self.x_sol + dx
                    new_y_sol = self.y_sol + dy

                    # Run the simulation with the new adjusted positions
                    self.x_sol = new_x_sol
                    self.y_sol = new_y_sol

                    # but only run it when the adjusted positions are within bounds
                    if not self.out_of_valid_region():
                        self.run_simulation()
                        self.eval_by_FD() # append fd for the try-outs into the fd_history, new sol_fd1, sol_fd2 generated

                        # Calculate criteria based on stopSignal
                        if self.stopSignal == 'exitPoint':
                            x_e, y_e = self.get_closest_point(self.simulated_path)
                            delta_y = abs(self.exit_point[1] - y_e)
                            results.append((delta_y, new_x_sol, new_y_sol, self.simulated_path))
                        elif self.stopSignal == 'pathSim':
                            fd_combined = self.sol_fd1 + self.sol_fd2
                            results.append((fd_combined, new_x_sol, new_y_sol, self.simulated_path))


                    else:
                        results.append((np.infty, new_x_sol, new_y_sol, None))

                    # reset values of self.x_sol, self.y_sol
                    self.x_sol, self.y_sol = original_x, original_y
                
                if self.debug:
                    print([item[0] for item in results])

                # Find the direction with the best outcome
                best_result = min(results, key=lambda r: r[0])
                best_dis, best_x_sol, best_y_sol, best_simulated_path = best_result

                # Check if the best direction improves upon the current solution
                if best_dis < opt_distance:
                    # update opt_distance
                    opt_distance = best_dis

                    # Set the best solution found in this iteration
                    self.x_sol = best_x_sol
                    self.y_sol = best_y_sol
                    self.simulated_path = best_simulated_path
                    self.n_steps += 1

                    # Call eval_by_FD to update sol_fd1 and sol_fd2 with the best solution found
                    # attach the best fd again (the fd for each actual adjustment will appear twice in the history)
                    self.eval_by_FD()

                    # Append history
                    if self.debug:
                        print(f'Appending better solution: {self.x_sol, self.y_sol}')
                        
                    self.sol_history.append([self.x_sol, self.y_sol])
                    self.simulated_path_history.append(self.simulated_path)

                    if self.debug:
                        print('Best Result After Adjustment')
                        print(best_result[0], best_result[1], best_result[2])
                else: 
                    # Stop if no improvement
                    # if no improvements, fd will not be appended to the history again
                    if self.debug:
                        print('No Better Adjustments')
                    break

                # Check if the stopping criteria are met
                if self.stop_adjust(epsilon):
                    break

        # Return all the information, including histories and last Frechet distances
        return {
            'x_sol': self.x_sol,
            'y_sol': self.y_sol,
            'n_steps': self.n_steps,
            'sol_history': self.sol_history,
            'fd_history': self.sol_fd_history, # might contain fd for all 8 directions
            'simulated_path_history': self.simulated_path_history
        }

    
    ### code for visualizing
    

    def visualize_adjustment_process(self, modelName, showPath = True):
        fig, ax = plt.subplots(figsize=(9, 6), dpi=200)
        ax.set_xlim(100, 900)
        ax.set_ylim(60, 540)
        ax.invert_yaxis()
        ax.set_aspect('equal')
        ax.set_title(f'Stimulus {self.stimulus_idx}, Obstacle {self.obstacle_idx}')
        
        print(self.sol_history)
    
        subset = drawInitialSetUp(ax, df_all, self.stimulus_idx, self.obstacle_idx)

        # Draw Participant Responses
        ax.scatter(subset["triangle_final_x_flipback"], subset["triangle_final_y"], s=50, marker = '+', color = '#0A704E', alpha = 1)

        # Human Average FD
        human_fd = subset['fd_combined'].mean()
#         print(human_fd)
#         print(self.sol_fd_history)
        
#         # Calculate normalized differences and map to colors with controlled scaling
#         fd_differences = np.abs(np.array(self.sol_fd_history) - self.subset['fd_combined'].mean())
#         if (fd_differences.max() - fd_differences.min()) != 0:
#                 normalized_differences = (fd_differences - fd_differences.min()) / (fd_differences.max() - fd_differences.min())
#         else:
#             # If all values are the same, set them to the middle of the scale (0.5 is arbitrary and can be adjusted)
#             normalized_differences = np.full_like(fd_differences, fill_value=0.5)
        
# #         print(normalized_differences)
#         # Define lower and upper bounds for the color scaling
#         lower_bound = 0
#         upper_bound = 0.6
#         scaled_differences = lower_bound + (upper_bound - lower_bound) * normalized_differences
        
#         colors = [plt.cm.YlOrBr(1 - diff) for diff in scaled_differences]  # Using scaled differences


        colors = [plt.cm.YlOrBr(index/len(self.sol_history)) for index, item in enumerate(self.sol_history)]
        
        # Initial placement for the moving obstacle
        obstacle_patch = patches.RegularPolygon(
            (self.sol_history[0][0], self.sol_history[0][1]), numVertices=3, radius=45, orientation=np.pi, color=colors[0], zorder=7
        )
        ax.add_patch(obstacle_patch)

        # Plot list of x markers and dashed lines (for cumulative path)
        cumulative_path_x = []
        cumulative_path_y = []

        lines = []  # Store line objects to keep all on plot

        def init():
            # Initialize with empty paths
            cumulative_path_x.clear()
            cumulative_path_y.clear()
            return obstacle_patch,

        def update(frame):
            # Update the obstacle position
            obstacle_patch.xy = (self.sol_history[frame][0], self.sol_history[frame][1])
            obstacle_patch.set_color(colors[frame])

            # Add trajectory lines
            # Specifically, darker lines indicate trajectories whose Fréchet distance ($fd$) is 
            # closer to the average Fréchet distance of human responses ($fd_{human}$) for that condition.
            if frame < len(self.simulated_path_history):
                simulated_path = np.array(self.simulated_path_history[frame])
                alpha = frame / len(self.sol_history)  # Normalize alpha value
                obstacle_patch.set_alpha(min(1, 0.5 + alpha * 0.5))
                line, = ax.plot(simulated_path[:, 0], simulated_path[:, 1], color=colors[frame], linewidth=2, alpha=min(1, 0.5 + alpha * 0.5), zorder=7)
                lines.append(line)

                if showPath:
                    # Append current position to the cumulative path
                    cumulative_path_x.append(self.sol_history[frame][0])
                    cumulative_path_y.append(self.sol_history[frame][1])

                    # Plot cumulative path as x-x-x-x
                    ax.plot(cumulative_path_x, cumulative_path_y, linestyle='-', marker='o', color='darkgray', markersize=2, linewidth=1, alpha=min(1, 0.5 + alpha * 0.5), zorder=7)
            return lines + [obstacle_patch]

        frames = len(self.sol_history)
        ani = FuncAnimation(fig, update, init_func=init, frames=frames, blit=True, repeat=False)

        # Add a global legend
        handles = [
            patches.Patch(color=ground_truth_color, label='Ground Truth'),
            patches.Patch(color=human_centroid_color, label='Human Centroid'),
            patches.Patch(color='#0A704E', label='Participant Responses'),
            patches.Patch(color=participant_color, label='Participant Trajectories'),
            patches.Patch(color=anchor_color, label='Anchor Position'),
            patches.Patch(color=plt.cm.YlOrBr(0.5), label='Model Prediction')
        ]

        # Add the legend outside the plot
        leg = fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=3, fontsize='10')

        from pathlib import Path

        # Create the directory
        animation_dir = Path(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_animation')
        animation_dir.mkdir(parents=True, exist_ok=True)

        # Save the animation
        ani.save(animation_dir / f'adjustment_animation_{self.stimulus_idx}-{self.obstacle_idx}.gif', writer='pillow', fps=1)
        plt.close(fig)  # Prevent double display in Jupyter Notebook

        # To display the animation in a Jupyter notebook
        from IPython.display import HTML
        return HTML(ani.to_jshtml())

A very sensible code explanation provided by ChatGPT (new):

### 1. **Initialization (`__init__` method)**:
   - The `SimulationScenario` class is designed to simulate a specific scenario where a ball interacts with an obstacle in a predefined environment.
   - When an instance is created, it takes in various parameters related to the simulation, such as the stimulus and obstacle indices, initial positions of obstacles, the initial position of the ball, exit points, initialization and stopping criteria, and maximum steps for adjustment.
   - It filters a dataset (`df_all`) to focus on a specific scenario based on the provided stimulus and obstacle indices, and it extracts the exact path for comparison purposes.

### 2. **Setting Initial Solution (`set_initial_solution` method)**:
   - This method initializes the x and y positions of the obstacle based on the selected initialization strategy (e.g., Linear Projection, Anchor, or Mid).
   - It includes a heuristic adjustment to ensure the initial y position is within screen bounds.

### 3. **Creating Directory and Running Simulation (`create_directory` and `run_simulation` methods)**:
   - `create_directory`: Creates a directory for storing simulation results based on the stimulus and obstacle indices.
   - `run_simulation`: Runs a simulation using an external JavaScript file (`simulateFunc.js`) and stores the results in a JSON file. If the file doesn’t exist, it creates one by running the simulation.

### 4. **Evaluating Path Similarity (`eval_by_FD` method)**:
   - This method calculates the similarity between the simulated path and the exact path using the Fréchet distance. It checks whether the simulated path closely follows the exact path based on entry and exit points.

### 5. **Stopping Criteria (`stop_adjust` method)**:
   - This method determines whether the adjustment process should stop based on several criteria:
     - If the x or y position of the obstacle is out of bounds.
     - If the solution has already been evaluated.
     - Based on the chosen stopping criteria (`exitPoint` or `pathSim`), it checks whether the simulation results are close enough to the expected outcome.

### 6. **Adjusting the Obstacle (`adjust_obstacle` method)**:
   - The main function of the class, this method iteratively adjusts the position of the obstacle to better align the simulated exit point with the actual exit point.
   - It evaluates different directions of adjustment (up, down, left, right, and diagonals) and selects the best one based on the simulation results.
   - The process continues until the stopping criteria are met or the maximum number of steps is reached. The method returns all relevant information, including the history of adjustments and path similarities.

### 7. **Visualization (`visualize_adjustment_process` method)**:
   - This method visualizes the adjustment process by creating an animation that shows how the obstacle’s position changes over time and how the simulated path evolves.
   - It generates a GIF of the adjustment process and displays it, making it easier to understand how the simulation progresses.

### **Summary**:
   - The code is part of a simulation tool that iteratively adjusts the position of an obstacle to achieve a desired outcome, specifically aligning the simulated path of a ball with an expected path.
   - It involves initializing the scenario, running simulations, evaluating the results, making adjustments, and visualizing the process.
   - The goal is to fine-tune the obstacle’s position using a systematic approach, guided by the chosen initialization and stopping criteria, to achieve the best possible alignment between the simulated and actual paths.

A very sensible code explanation provided by ChatGPT (previous):

The adjustment process in your `SimulationScenario` class involves modifying the position of an obstacle in a simulated environment to align the simulated exit point of a ball with its actual exit point. The process iterates through potential positions of the obstacle, adjusting based on the difference between the current simulated exit and the actual exit point, while respecting the conditions imposed by the stopping criteria.

### Explanation of the Adjustment Process

1.  **Initialization**: The initial position of the obstacle is set based on predefined initial conditions (`x_init` and `y_init`). This might involve different strategies like midpoint initialization or based on specific anchor points.
    
2.  **Running Simulation**: The obstacle's initial position is used to run a physics simulation (presumably via a JavaScript Node.js script) to see where a ball would exit if it interacted with the obstacle positioned at those coordinates.
    
3.  **Adjustment Loop**:
    
    *   The simulation's exit point is compared to the actual exit point.
    *   If the simulated exit point (vertical position `y_e`) is less than the actual exit point (`y_exit`), the obstacle is moved downwards, otherwise, it is moved upwards by a step size `d`.
    *   This adjustment is repeated, modifying the obstacle's position incrementally and re-running the simulation after each adjustment.
4.  **Stopping Criteria**:
    
    *   The adjustment loop can stop if the vertical position of the obstacle moves out of predefined screen bounds.
    *   It can also stop if the obstacle's adjusted position repeats (cyclic adjustments indicating a local minimum or a configuration that doesn’t converge).
    *   Currently, it stops if the vertical distance to the actual exit point is less than the threshold `epsilon`, indicating the simulated exit is sufficiently close to the actual exit.

In [ ]:
print(positions)
print(anchors)
print(ball_Xs)

## Function for running the whole simulation

In [143]:
def find_best_parameters(df_all, positions, anchors, ball_Xs, x_blue_range, exitPoints_single, 
                         d_values, epsilon_values, max_adjustments, 
                         x_init, y_init, init_choice, stop_signal, debug):
    
    # Initialize dict for storing the single best-fit parameters & history
    best_params = {
        'd': None,
        'epsilon': None,
        'maxSteps': None,
        'best_ml': -np.inf,
        'best_cov': None,
        'best_x': [],
        'best_y': [],
        'adjustment_histories': [],
        'best_n_steps': None
    }
    
    for d in d_values:
        print(f'TESTING d = {d}...')
        for epsilon in epsilon_values:
            print(f'TESTING epsilon = {epsilon}...')
            for maxSteps in max_adjustments:
                print(f'TESTING max_steps = {maxSteps}...')
                x_sols, y_sols, n_steps, histories = [], [], [], []
                for stimulus_idx in positions:
                    for obstacle_idx in anchors:
                        # Initialize the simulation scenario
                        simulation_scenario = SimulationScenario(
                            df_all, stimulus_idx, obstacle_idx, x_init, y_init,
                            ball_Xs, x_blue_range, exitPoints_single, init_choice, stop_signal, maxSteps, debug
                        )
                        # Run the adjustment process
                        # result = simulation_scenario.adjust_obstacle(d, epsilon)
                        try:
                            result = simulation_scenario.adjust_obstacle(d, epsilon)
                            # print(f"for {stimulus_idx}, {obstacle_idx}, x_sol is {result['x_sol']}")
                            x_sols.append(result['x_sol'])
                            y_sols.append(result['y_sol'])
                            n_steps.append(result['n_steps'])
                            histories.append({
                                'sol_history': result['sol_history'],
                                'fd_history': result['fd_history'],
                                'simulated_path_history': result['simulated_path_history']
                            })
                            print(f'Done with {stimulus_idx}-{obstacle_idx}')
                        except Exception as e:
                            print(f"Error encountered with stimulus_idx {stimulus_idx}, obstacle_idx {obstacle_idx}: {e}")
                            print(result['x_sol'], result['y_sol'])
                            continue
#                         print('--------------------------------------')
                        # if np.random.random() < 0.00001:
                        #     print('Visualization')
                        #     simulation_scenario.visualize_adjustment_process(modelName)

                if len(x_sols) == 12 and len(y_sols) == 12:
                    print('Correcting Size Mismatch...')
                    x_sols = [x_sol for x_sol in x_sols for _ in range(6)]
                    y_sols = [y_sol for y_sol in y_sols for _ in range(6)]
                # Evaluate the model, assume find_best_cov_model2 is a function that returns (cov, ml)
                if len(x_sols) == len(y_sols) and len(x_sols) == 72:
                    print('Evaluating MLL')
                    cov, ml = find_best_cov_model2(x_sols, y_sols, df_all)  # Example function
                else: 
                    print(f'MLL not Evaluated!!! ERROR {d}, {epsilon}, {maxSteps}')
                
                # Update the best parameters if the new model likelihood is better
                print('Current MLL, New MLL:')
                print(ml, best_params['best_ml'])
                if ml > best_params['best_ml']:
                    best_params.update({
                        'd': d,
                        'epsilon': epsilon,
                        'maxSteps': maxSteps,
                        'best_ml': ml,
                        'best_cov': cov,
                        'best_x': x_sols,
                        'best_y': y_sols,
                        'adjustment_histories': histories,
                        'best_n_steps':n_steps
                    })
                    
                print(f'DONE TESTING max_steps = {maxSteps}')
                print('-------------------------------------')
            print(f'DONE TESTING epsilon = {epsilon}')
            model_dir = Path(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}')
            model_dir.mkdir(parents=True, exist_ok=True)
            # when not under debug mode, save results
            if not debug:
                with open(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_results_covering_{d}_{epsilon}_{maxSteps}.pkl', 'wb') as f:
                    pickle.dump(best_params, f)
            print('***************************************')
        print(f'DONE TESTING d = {d}')
        print('+++++++++++++++++++++++++++++++++++++++')

    return best_params

# Usage of the function
# positions = range(1, 13)  # Stimulus indices
# anchors = range(1, 7)     # Obstacle indices
# d_values = [1, 5, 10, 20, 40]  # Displacement magnitudes
# epsilon_values = [1, 5, 10, 20, 40]  # Tolerance levels
# max_adjustments = [np.inf]  # Maximum adjustment steps

# Initialize your x_init and y_init based on your specific needs
# Example: Mid+Sim
# x_init = [[intersections_modified_by_anchor[(stimulus_idx, obstacle_idx)][0] for obstacle_idx in anchors] for stimulus_idx in positions]
# y_init = [[300]*6]*12

# Run the function
# best_setup = find_best_parameters(df_all, positions, anchors, ball_Xs, x_blue_range, exitPoints_single,
#                                   d_values, epsilon_values, max_adjustments, x_init, y_init, 'Mid', 'exitPoint')

# The best_setup dictionary contains the best parameters, model likelihood, and comprehensive adjustment histories.

## Defining the grid for free parameters

In [144]:
# Usage of the function
positions = range(1, 13)  # Stimulus indices
anchors = range(1, 7)     # Obstacle indices
d_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]  # Displacement magnitudes
# Sam's rec: A shift of 20-40 pixels is pretty large, so it's not surprising that the optimal fit never chooses those values for d. 
# I think it probably makes sense to restrict the range to 1-10 and include more values within that range.
epsilon_values = [1, 5, 10, 20, 40]  # Tolerance levels # Exit Point
# epsilon_values = [10, 20, 40, 80, 100, 150, 200] # Path Sim
max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]  # Maximum adjustment steps

In [145]:
def storeBestParams(best_setup, modelName):
    storeXYPrediction(best_setup['best_x'], best_setup['best_y'], modelName)

    with open(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_results.pkl', 'wb') as f:
        pickle.dump(best_setup, f)

def runEvalsFigs(modelName, df_all, n_free_params, withAdjust = False, best_setup = None):
    result_file_path = f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json'
    if not os.path.exists(result_file_path):
        runScript_72Simulations(f'runSimulation/full72Trajectories/{modeling_choice}/run72Simulations.py', modelName)
    with open(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_results.pkl', 'rb') as f:
        best_setup = pickle.load(f)
    getBICFromPrediction(best_setup['best_x'], best_setup['best_y'], df_all, n_free_params)
    compareFD_model_human(result_file_path, df_all, modelName)
    visualize72Trials(result_file_path, df_all, modelName, withAdjust, best_setup)
    # visualize72Trials(result_file_path, df_all)
    return best_setup

#### if compare with a random parameter set (for results checking & debugging)

In [146]:
def debugResults(d, e, mxSteps, oneAnchorOrAll, x_init, y_init, modelInit, modelStop):
    debug_results = find_best_parameters(df_all, positions, oneAnchorOrAll, ball_Xs, x_blue_range, exitPoints_single,
                                            [d], [e], [mxSteps], x_init, y_init, modelInit, modelStop, True)
    return debug_results

## LP + Sim

In [88]:
# best_setup_LP_Sim = runEvalsFigs('LP+Sim', df_all, 4)

In [370]:
# x_model = [nearest_value_in_range(intersections[stimulus_idx][0],x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for stimulus_idx in positions for obstacle_idx in anchors] #(1,1)(1,2)(1,3)...(1,6)(2,1)
# y_model = [nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for stimulus_idx in positions for obstacle_idx in anchors]
# for i in range(12):
#     for j in range(6):
#         if not y_init[i][j] == y_model[i*6+j]:
#             print(y_init[i][j])
#             print(y_model[i*6+j])
#             print('Something Wrong')

In [89]:
modelName = 'LP+Sim'
max_adjustments = [np.infty]

# Initialize your x_init and y_init based on your specific needs

x_init = [[nearest_value_in_range(intersections[stimulus_idx][0], x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_LP_Sim = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                         d_values, epsilon_values, max_adjustments, x_init, y_init, 'LP', 'exitPoint', False)

storeBestParams(best_setup_LP_Sim, modelName)

In [ ]:
modelName = 'LP+Sim'

storeXYPrediction(best_setup_LP_Sim['best_x'], best_setup_LP_Sim['best_y'], modelName)

with open(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_results.pkl', 'wb') as f:
    pickle.dump(best_setup_LP_Sim, f)

In [94]:
modelName = 'LP+Sim'

with open(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_results.pkl', 'rb') as f:
    best_setup_LP_Sim = pickle.load(f)

In [ ]:
best_setup_LP_Sim

In [ ]:
# Test visualization
simulation_scenario_test = SimulationScenario(
                            df_all, 2, 1, x_init, y_init,
                            ball_Xs, x_blue_range, exitPoints_single, 'LP', 'exitPoint', np.infty, True)
result = simulation_scenario_test.adjust_obstacle(15, 1)
simulation_scenario_test.visualize_adjustment_process(modelName)

In [ ]:
getBICFromPrediction(best_setup_LP_Sim['best_x'], best_setup_LP_Sim['best_y'], df_all, 4)

### FD evaluations

In [ ]:
modelName = 'LP+Sim'
# runScript_72Simulations('runSimulation/full72Trajectories/run72Simulations.py', modelName)
# compareFD_model_human(f'runSimulation/full72Trajectories/{modelName}/all72_results.json', df_all, modelName)
# visualize72Trials(f'runSimulation/full72Trajectories/{modelName}/all72_results.json',df_all)
best_setup_LP_Sim = runEvalsFigs('LP+Sim', df_all, 4)

### Check Results

In [ ]:
# check Results
x_init = [[nearest_value_in_range(intersections[stimulus_idx][0], x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

debug_results = debugResults(15, 1, np.infty, [1], x_init, y_init, 'LP', 'exitPoint')

In [ ]:
debug_results['best_ml']

## LP + Sim_max

In [ ]:
modelName = 'LP+Sim_max'
max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(intersections[stimulus_idx][0], x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_LP_Sim_max = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'LP', 'exitPoint', False)

# Store the results
storeBestParams(best_setup_LP_Sim_max, modelName)

In [98]:
modelName = 'LP+Sim_max'

with open(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_results.pkl', 'rb') as f:
    best_setup_LP_Sim_max_debug = pickle.load(f)

In [ ]:
best_setup_LP_Sim_max = runEvalsFigs('LP+Sim_max', df_all, 5)
print(best_setup_LP_Sim_max['best_x'])
print(best_setup_LP_Sim_max['best_y'])

In [ ]:
getBICFromPrediction(best_setup_LP_Sim_max['best_x'], best_setup_LP_Sim_max['best_y'], df_all, 5)

In [ ]:
# Test visualization
simulation_scenario_test = SimulationScenario(
                            df_all, 1, 1, x_init, y_init,
                            ball_Xs, x_blue_range, exitPoints_single, 'LP', 'exitPoint', 9)
result = simulation_scenario_test.adjust_obstacle(8, 10)
simulation_scenario_test.visualize_adjustment_process(modelName)

## Anchor + Sim

In [ ]:
modelName = 'Anchor+Sim'
# max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
max_adjustments = [np.infty]
# anchors = np.sort(df_all['obstacle_idx'].unique())

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(anchor_Xs[stimulus_idx][obstacle_idx],x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions]
y_init = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_initial_y'].first() 

# Run the function
best_setup_Anchor_sim = find_best_parameters(df_all, positions, anchors, ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Anchor', 'exitPoint', False)

# Store the results
storeBestParams(best_setup_Anchor_sim, modelName)

In [ ]:
best_setup_Anchor_sim

In [ ]:
best_setup_Anchor_sim = runEvalsFigs('Anchor+Sim', df_all, 4, True, best_setup_Anchor_sim)
print(best_setup_Anchor_sim['d'],best_setup_Anchor_sim['epsilon'])

## Anchor + Sim_max

In [ ]:
modelName = 'Anchor+Sim_max'
max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
# max_adjustments = [np.infty]
# anchors = np.sort(df_all['obstacle_idx'].unique())

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(anchor_Xs[stimulus_idx][obstacle_idx],x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions]
y_init = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_initial_y'].first() 

# Run the function
best_setup_Anchor_sim_max = find_best_parameters(df_all, positions, anchors, ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Anchor', 'exitPoint', False)

# Store the results
storeBestParams(best_setup_Anchor_sim_max, modelName)

In [ ]:
best_setup_Anchor_sim_max

In [ ]:
modelName = 'Anchor+Sim_max'

with open(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/adjustment_results.pkl', 'rb') as f:
    best_setup_Anchor_sim_max = pickle.load(f)
best_setup_Anchor_sim_max = runEvalsFigs(modelName, df_all, 5, True, best_setup_Anchor_sim_max)
print(best_setup_Anchor_sim_max['d'],best_setup_Anchor_sim_max['epsilon'], best_setup_Anchor_sim_max['maxSteps'])

In [ ]:
# Test visualization
modelName = 'Anchor+Sim_max'
x_init = [[intersections_modified_by_anchor[(stimulus_idx,obstacle_idx)][0] for obstacle_idx in anchors] for stimulus_idx in positions]
y_init = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_initial_y'].first()
simulation_scenario_test = SimulationScenario(
                            df_all, 2, 1, x_init, y_init,
                            ball_Xs, x_blue_range, exitPoints_single, 'Anchor', 'exitPoint', 3)
result = simulation_scenario_test.adjust_obstacle(10, 10)
simulation_scenario_test.visualize_adjustment_process(modelName)

## Midline+Sim

In [ ]:
modelName = 'Mid+Sim'
# max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
max_adjustments = [np.infty]

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]


# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_mid_sim = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Mid', 'exitPoint', False)

# Store the results
storeBestParams(best_setup_mid_sim, modelName)

In [ ]:
best_setup_mid_sim

In [ ]:
best_setup_mid_sim = runEvalsFigs('Mid+Sim', df_all, 4)
print(best_setup_mid_sim['d'], best_setup_mid_sim['epsilon'])

In [ ]:
# check Results
modelName = 'Mid+Sim'
x_init = [[nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]
debug_results = debugResults(4, 10, np.infty, [1], x_init, y_init, 'Mid', 'exitPoint')

## Mid + Sim_max

In [ ]:
modelName = 'Mid+Sim_max'
max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
# max_adjustments = [np.infty]

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]


# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_mid_sim_max = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Mid', 'exitPoint', False)

# Store the results
storeBestParams(best_setup_mid_sim_max, modelName)

In [ ]:
best_setup_mid_sim_max['best_n_steps']
#  'd': 4,
#  'epsilon': 5,
#  'maxSteps': 10,
#  'best_ml': -14799.718164780481,
#  [9, 6, 2, 1, 2, 4, 1, 5, 1, 10, 2, 10]

In [ ]:
best_setup_mid_sim_max = runEvalsFigs('Mid+Sim_max', df_all, 5)
print(best_setup_mid_sim_max['d'], best_setup_mid_sim_max['epsilon'], best_setup_mid_sim_max['maxSteps'])

In [ ]:
visualize72Trials(f'runSimulation/full72Trajectories/{modeling_choice}/{modelName}/all72_results.json', df_all, modelName, False, best_setup_mid_sim_max)

In [ ]:
# Test visualization
modelName = 'Mid+Sim_max'
x_init = [[intersections_modified_by_anchor[(stimulus_idx,obstacle_idx)][0] for obstacle_idx in anchors] for stimulus_idx in positions]
y_init = [[300]*6]*12
simulation_scenario_test = SimulationScenario(
                            df_all, 2, 1, x_init, y_init,
                            ball_Xs, exitPoints_single, 'Mid', 'exitPoint', 15)
result = simulation_scenario_test.adjust_obstacle(1, 20)
simulation_scenario_test.visualize_adjustment_process(modelName)

In [ ]:
# generate adjustment gifs
modelName = 'Mid+Sim_max'
simulation_scenario_test = SimulationScenario(
                            df_all, 5, 2, x_init, y_init,
                            ball_Xs, x_blue_range, exitPoints_single, 'Mid', 'exitPoint', 10, True)
result = simulation_scenario_test.adjust_obstacle(4, 5)
simulation_scenario_test.visualize_adjustment_process(modelName)

In [ ]:
# check Results
modelName = 'Mid+Sim_max'
x_init = [[nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]
debugged_results = debugResults(4, 5, 10, [1], x_init, y_init, 'Mid', 'exitPoint')

In [ ]:
runEvalsFigs(modelName, df_all, 5, False, debugged_results)

# Path Sim

## LP + Sim_ps

In [ ]:
modelName = 'LP+Sim_ps'
max_adjustments = [np.infty]
# max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
epsilon_values = [10, 20, 40, 80, 100, 150, 200]  # Tolerance levels, pathSim: entry+exit path similarity

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(intersections[stimulus_idx][0], x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_LP_Sim_ps = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'LP', 'pathSim', False)

# Store the results
storeBestParams(best_setup_LP_Sim_ps, modelName)

In [ ]:
best_setup_LP_Sim_ps
# 'd': 15,
#  'epsilon': 10,
#  'maxSteps': inf,
#  'best_ml': -15544.396347228148,

In [ ]:
best_setup_LP_Sim_ps = runEvalsFigs('LP+Sim_ps', df_all, 4)
print(best_setup_LP_Sim_ps['d'], best_setup_LP_Sim_ps['epsilon'])

## LP + Sim_ps_max

In [ ]:
modelName = 'LP+Sim_ps_max'
# max_adjustments = [np.infty]
max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
epsilon_values = [10, 20, 40, 80, 100, 150, 200]  # Tolerance levels, pathSim: entry+exit path similarity

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(intersections[stimulus_idx][0], x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_LP_Sim_ps_max = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'LP', 'pathSim', False)

# Store the results
storeBestParams(best_setup_LP_Sim_ps_max, modelName)

In [ ]:
best_setup_LP_Sim_ps_max = runEvalsFigs('LP+Sim_ps_max', df_all, 5)
print(f"d: {best_setup_LP_Sim_ps_max['d']}, epsilon: {best_setup_LP_Sim_ps_max['epsilon']}, maxSteps: {best_setup_LP_Sim_ps_max['maxSteps']}")

In [ ]:
best_setup_LP_Sim_ps_max
#  'd': 20,
#  'epsilon': 10,
#  'maxSteps': 4,
#  'best_ml': -15478.41455064393,

In [ ]:
# try alternative combination & test if best
modelName = 'LP+Sim_ps_max_debug'
x_init = [[nearest_value_in_range(intersections[stimulus_idx][0], x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(intersections[stimulus_idx][1], 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

debuged_results = debugResults(20, 40, 4, [1], x_init, y_init, 'LP', 'pathSim')

In [ ]:
debuged_results['best_x']

## Anchor + Sim_ps

In [ ]:
modelName = 'Anchor+Sim_ps'
# anchors = np.sort(df_all['obstacle_idx'].unique())
max_adjustments = [np.infty]
# max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
epsilon_values = [10, 20, 40, 80, 100, 150, 200]

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(anchor_Xs[stimulus_idx][obstacle_idx],x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions]
y_init = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_initial_y'].first()

# Run the function
best_setup_Anchor_sim_ps = find_best_parameters(df_all, positions, anchors, ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Anchor', 'pathSim', False)

# Store the results
storeBestParams(best_setup_Anchor_sim_ps, modelName)

In [ ]:
best_setup_Anchor_sim_ps = runEvalsFigs(modelName, df_all, 4)
print(best_setup_Anchor_sim_ps['d'], best_setup_Anchor_sim_ps['epsilon'])

In [ ]:
best_setup_Anchor_sim_ps
#  'd': 20,
#  'epsilon': 10,
#  'maxSteps': inf,
#  'best_ml': -14931.346174071901

## Anchor + Sim_ps_max

In [ ]:
modelName = 'Anchor+Sim_ps_max'
# anchors = np.sort(df_all['obstacle_idx'].unique())
# max_adjustments = [np.infty]
max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
epsilon_values = [10, 20, 40, 80, 100, 150, 200]

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(anchor_Xs[stimulus_idx][obstacle_idx],x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions]
y_init = df_all.groupby(['stimulus_idx','obstacle_idx'])['obstacle_initial_y'].first()


# Run the function
best_setup_Anchor_sim_ps_max = find_best_parameters(df_all, positions, anchors, ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Anchor', 'pathSim', False)

# Store the results
storeBestParams(best_setup_Anchor_sim_ps_max, modelName)

In [ ]:
best_setup_Anchor_sim_ps_max

In [ ]:
best_setup_Anchor_sim_ps_max = runEvalsFigs('Anchor+Sim_ps_max', df_all, 5, True, best_setup_Anchor_sim_ps_max)
print(f"d: {best_setup_Anchor_sim_ps_max['d']}, epsilon: {best_setup_Anchor_sim_ps_max['epsilon']}, maxSteps: {best_setup_Anchor_sim_ps_max['maxSteps']}")


## Mid + Sim_ps

In [ ]:
modelName = 'Mid+Sim_ps'
max_adjustments = [np.infty]
# max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
epsilon_values = [10, 20, 40, 80, 100, 150, 200]

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_mid_sim_ps = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Mid', 'pathSim', False)

# Store the results
storeBestParams(best_setup_mid_sim_ps, modelName)

In [ ]:
best_setup_mid_sim_ps
#  'd': 20,
#  'epsilon': 10,
#  'maxSteps': inf,
#  'best_ml': -14819.879827791834

In [ ]:
best_setup_mid_sim_ps = runEvalsFigs(modelName, df_all, 4)
print(best_setup_mid_sim_ps['d'], best_setup_mid_sim_ps['epsilon'])

In [ ]:
# test the adjustment history
with open('runSimulation/full72Trajectories/Model_both_x_and_y/Mid+Sim_ps/adjustment_results.pkl', 'rb') as file:
    adjustment_his = pickle.load(file)

for condition in adjustment_his['adjustment_histories']:
    print(condition['sol_history'][-1])

In [ ]:
best_setup_mid_sim_ps['best_y'] # barely adjusted (midline without adjustment)

# Mid + Sim_ps_max

In [ ]:
modelName = 'Mid+Sim_ps_max'
# max_adjustments = [np.infty]
max_adjustments = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
epsilon_values = [10, 20, 40, 80, 100, 150, 200]

# Initialize your x_init and y_init based on your specific needs
x_init = [[nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

# Run the function
# Results will be the same across different anchors, so can just run one anchor
best_setup_mid_sim_ps_max = find_best_parameters(df_all, positions, [1], ball_Xs, x_blue_range, exitPoints_single,
                                  d_values, epsilon_values, max_adjustments, x_init, y_init, 'Mid', 'pathSim', False)

# Store the results
storeBestParams(best_setup_mid_sim_ps_max, modelName)

In [ ]:
best_setup_mid_sim_ps_max

In [ ]:
best_setup_mid_sim_ps_max = runEvalsFigs('Mid+Sim_ps_max', df_all, 5)
print(best_setup_mid_sim_ps_max['d'], best_setup_mid_sim_ps_max['epsilon'], best_setup_mid_sim_ps_max['maxSteps'])

In [ ]:
best_setup_mid_sim_ps_max['best_x'][0]

In [ ]:
# Test visualization
modelName = 'Mid+Sim_ps_max'
x_init = [[nearest_value_in_range(canvasWidth/2, x_blue_range[(stimulus_idx,obstacle_idx)][0],x_blue_range[(stimulus_idx,obstacle_idx)][1]) for obstacle_idx in anchors] for stimulus_idx in positions] #(1,1)(1,2)(1,3)...(1,6)(2,1)
y_init = [[nearest_value_in_range(canvasHeight/2, 150 + margin, 450 - margin) for obstacle_idx in anchors] for stimulus_idx in positions]

simulation_scenario_test = SimulationScenario(
                            df_all, 2, 1, x_init, y_init,
                            ball_Xs, exitPoints_single, 'Mid', 'pathSim', 3)
result = simulation_scenario_test.adjust_obstacle(5, 10)
simulation_scenario_test.visualize_adjustment_process(modelName)

## Visualizations with adjustment

In [112]:
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.colorbar import ColorbarBase

# new function with legend
def visualize72Trials(jsonFilePath, df_all, modelName, withAdjust = False, best_setup = None):
    # Load JSON data, containing simulated trajectories from model prediction
    with open(jsonFilePath, 'r') as file:
        data = json.load(file)

    # Convert data to DataFrame
    df = pd.json_normalize(data)

    # Create a 12x6 grid of plots with shared axes and constrained layout
    fig, axes = plt.subplots(nrows=12, ncols=6, figsize=(30, 40), sharex=True, sharey=True, constrained_layout=True)

    # Loop through each row in the dataframe
    for idx, row in df.iterrows():
        stimulus_idx = row['stimulus_idx'] - 1  # 0-based index for matplotlib
        obstacle_idx = row['obstacle_idx'] - 1  # 0-based index

        ax = axes[stimulus_idx, obstacle_idx]
        ax.set_aspect('equal')
        
        if 'LP' in modelName:
            intersection = drawProjectedPath(row['stimulus_idx'], df_all, ax, 7)
        
        subset = drawInitialSetUp(ax, df_all, row['stimulus_idx'], row['obstacle_idx'])
        
        if withAdjust:
            # Find history for this specific condition
            history_idx = (row['stimulus_idx'] - 1) * 6 + (row['obstacle_idx'] - 1)
            history = best_setup['adjustment_histories'][history_idx]
            
            # Human Average FD
            human_fd = subset['fd_combined'].mean()
#             print(human_fd)
            
            fd_history = history['fd_history']
            
            # add before/after adjustment
            initial_x, initial_y = history['sol_history'][0][0],history['sol_history'][0][1]
            final_x, final_y = best_setup['best_x'][history_idx],best_setup['best_y'][history_idx]
            ax.text(0.95, 0.95, f'Initial: ({initial_x:.1f}, {initial_y:.1f})\nFinal: ({final_x:.1f}, {final_y:.1f})',
            verticalalignment='top', horizontalalignment='right',
            transform=ax.transAxes, color='dimgray', fontsize=8)
        
            # Calculate normalized differences and map to colors with controlled scaling
            fd_differences = np.abs(np.array(fd_history) - human_fd)
            if (fd_differences.max() - fd_differences.min()) != 0:
                normalized_differences = (fd_differences - fd_differences.min()) / (fd_differences.max() - fd_differences.min())
            else:
                # If all values are the same, set them to the middle of the scale (0.5 is arbitrary and can be adjusted)
                normalized_differences = np.full_like(fd_differences, fill_value=0.5)
#             normalized_differences = fd_differences / 100
    
#             print(row['stimulus_idx'], row['obstacle_idx'])
#             print(normalized_differences)
            # Define lower and upper bounds for the color scaling
            lower_bound = 0
            upper_bound = 0.6
            scaled_differences = lower_bound + (upper_bound - lower_bound) * normalized_differences
        
            colors = [plt.cm.YlOrBr(1 - diff) for diff in scaled_differences]  # The darker the color, the closer to fd_human_avg
            
            # Check if row['obstacle_X'] == best_setup['best_x'][history_idx]
            error = 0.00001
            if (abs(row['obstacle_X']-best_setup['best_x'][history_idx]) >= error) or (abs(row['obstacle_Y']-best_setup['best_y'][history_idx]) >= error):
#                 print(row['obstacle_X'], best_setup['best_x'][history_idx])
#                 print(row['obstacle_Y'], best_setup['best_y'][history_idx])
                raise Exception('Unmatched Data. Check history_index with df_all.')
                
            obstacle = patches.RegularPolygon((row['obstacle_X'], row['obstacle_Y']), numVertices=3, radius=45, orientation=np.pi, color=colors[-1], fill=True, alpha=0.7)
            ax.add_patch(obstacle)
            
            # Plot adjustment histories
            for i, step in enumerate(history['simulated_path_history']):
                step = np.array(step)
#                 alpha = i/len(history['simulated_path_history'])
                alpha = 1
                ax.plot(step[:, 0], step[:, 1], '-', alpha=alpha, zorder=7, color=colors[i], linewidth=1)  # Change color and alpha as needed
                
        else:
            # Add the model predicted obstacle position
            obstacle = patches.RegularPolygon((row['obstacle_X'], row['obstacle_Y']), numVertices=3, radius=45, orientation=np.pi, color=model_prediction_color, fill=True, alpha=0.7)
            ax.add_patch(obstacle)

            # Plot trajectory
            x_coords = [point[0] for point in row['simulated_trial']]
            y_coords = [point[1] for point in row['simulated_trial']]
            ax.plot(x_coords, y_coords, '-', linewidth=1.5, color='darkblue', label='Trajectory', zorder=6)

        ax.set_xlim(100, 900)
        ax.set_ylim(60, 540)
        ax.invert_yaxis()
        ax.set_title(f'Stimulus {row["stimulus_idx"]}, Obstacle {row["obstacle_idx"]}')

    # Add a single global legend
    handles = [
        patches.Patch(color=ground_truth_color, label='Ground Truth'),
        patches.Patch(color=human_centroid_color, label='Human Centroid'),
        patches.Patch(color=participant_color, label='Participant Trajectories'),
        patches.Patch(color=anchor_color, label='Anchor Position')
    ]
    if withAdjust:
        handles.append(patches.Patch(color=plt.cm.YlOrBr(0.5), label='Model Prediction'))
    else:
        handles.append(patches.Patch(color=model_prediction_color, label='Model Prediction'))
        
    leg = fig.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5, 1.02), ncol=5, fontsize='16')
    
    fig.supxlabel('X Coordinate', fontsize=16)
    fig.supylabel('Y Coordinate', fontsize=16)
    plt.savefig(f'{modelName}_72Trials.pdf', bbox_inches='tight', bbox_extra_artists=(leg,))
    plt.show()

# This code assumes you have the jsonFilePath and df_all prepared
# visualize72Trials('path_to_your_data.json', df_all_dataframe)

In [ ]:
runEvalsFigs('Mid+Sim_ps_max', df_all, 5, True, best_setup_mid_sim_ps_max)

## Baseline FD comparison human vs. human

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

# Assuming df_all contains a column 'fd_combined' with each participant's data
correlations = []

# Repeat the process 30 times
for _ in range(30):
    group_correlations = []
    group_stds = []
    
    # Group by 'stimulus_idx' and 'obstacle_idx'
    grouped = df_all.groupby(['stimulus_idx', 'obstacle_idx'])
    
    for _, group in grouped:
        # Shuffle and split the group into two halves
        shuffled_group = group.sample(frac=1, random_state=np.random.randint(0, 10000))

        # print(shuffled_group)
        half_size = len(shuffled_group) // 2
        # print(half_size)
        
        # Split into two halves
        first_half = shuffled_group.iloc[:half_size]['fd_combined']
        # print(shuffled_group.iloc[:half_size]['stimulus_idx'])
        # print(shuffled_group.iloc[:half_size]['obstacle_idx'])
        # print(shuffled_group.iloc[:half_size]['fd2_exit'])
        second_half = shuffled_group.iloc[half_size:]['fd_combined']
        
        # Calculate the mean for each half
        mean_first_half = first_half.mean()
        mean_second_half = second_half.mean()
        std_first_half = first_half.std()
        std_second_half = second_half.std()
        
        # Store the mean values as an (x, y) pair
        group_correlations.append((mean_first_half, mean_second_half))
        group_stds.append((std_first_half, std_second_half))
    
    # Now calculate the correlation across all (x, y) pairs
    if group_correlations:  # Ensure there are valid (x, y) pairs
        first_half_vals, second_half_vals = zip(*group_correlations)
        first_half_vals_std, second_half_vals_std = zip(*group_stds)


        correlation = plotComparison_fd(first_half_vals, second_half_vals, first_half_vals_std, 'Human_50', second_half_vals_std)
        correlations.append(correlation)

# Convert correlations list to a DataFrame or display directly
correlations_df = pd.DataFrame(correlations, columns=['Correlation'])
print(correlations_df)

In [ ]:
correlations_df.mean()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Data for models and corresponding BIC values
models = [
    "Uni Prior", "LP", "Anchor", "Midline",
    "LP+Exit Point", "Anchor+Exit Point", "Midline+Exit Point",
    "LP+Path Sim", "Anchor+Path Sim", "Midline+Path Sim"
]

bic_values = [
    31531.85, 32375.67, 31126.21, 30212.39,
    31664.11, 30084.29, 29621.91,
    30734.55, 29960.12, 29664.86
]

# Define color palette
palette = sns.color_palette("Set2", 4)  # 4 colors for diversity
colors = [
    'black', palette[0], palette[1], palette[2],  # No Adjustment models
    palette[0], palette[1], palette[2],              # Exit Point models
    palette[0], palette[1], palette[2]               # Path Sim models
]

# Define textures for different stopping criteria
textures = [
    '', '', '', '',       # No texture for no-adjustment models
    '//', '//', '//',        # '-' texture for Exit Point models
    '+', '+', '+'      # '\' texture for Path Sim models
]

# Assign edge colors to match fill colors
edge_colors = colors.copy()

# Define fill colors: white for no-adjustment models, colored for adjusted models
fill_colors = [
    'white' if '+' not in model else colors[i] 
    for i, model in enumerate(models)
]

# Create the plot
fig, ax = plt.subplots(figsize=(7, 6))

# Plot each bar with corresponding properties
for i in range(len(models)):
    ax.bar(
        i, 
        bic_values[i], 
        color=fill_colors[i], 
        edgecolor=edge_colors[i], 
        hatch=textures[i], 
        alpha=0.7, 
        linewidth=2
    )

# Set labels and title
ax.set_ylabel('BIC Values', fontsize=16)
# ax.set_title('BIC Values by Model', fontsize=16)
ax.set_xticks(np.arange(len(models)))
ax.set_xticklabels(models, rotation=45, ha="right", fontsize=12)

# Adjust y-axis limits for better visibility
y_min = min(bic_values) * 0.98
y_max = max(bic_values) * 1.02
ax.set_ylim(y_min, y_max)

# Create custom legend
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor='white', edgecolor='black', label='UniPrior Init'),
    Patch(facecolor=palette[0], edgecolor=palette[0], label='LP Init'),
    Patch(facecolor=palette[1], edgecolor=palette[1], label='Anchor Init'),
    Patch(facecolor=palette[2], edgecolor=palette[2], label='Midline Init'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='//', label='Exit Point'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='+', label='Path Sim')
]

ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

# Enhance layout
plt.tight_layout()

# Display the plot

plt.savefig("bic_values_by_model.png",dpi=300,bbox_inches='tight')
plt.show()

# updating values

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Data for models and corresponding BIC values
models = [
    "Uni Prior", "LP", "Anchor", "Midline",
    "LP+Exit Point", "Anchor+Exit Point", "Midline+Exit Point",
    "LP+Path Sim", "Anchor+Path Sim", "Midline+Path Sim"
]

bic_values = [
    31220.49, 32375.67, 31126.21, 30212.39,
    31642.99, 30119.87, 29635.80,
    30993.19, 29877.71, 29662.83
]

# Define color palette
palette = sns.color_palette("Set2", 4)  # 4 colors for diversity
colors = [
    'black', palette[0], palette[1], palette[2],  # No Adjustment models
    palette[0], palette[1], palette[2],              # Exit Point models
    palette[0], palette[1], palette[2]               # Path Sim models
]

# Define textures for different stopping criteria
textures = [
    '', '', '', '',       # No texture for no-adjustment models
    '//', '//', '//',        # '-' texture for Exit Point models
    '+', '+', '+'      # '\' texture for Path Sim models
]

# Assign edge colors to match fill colors
edge_colors = colors.copy()

# Define fill colors: white for no-adjustment models, colored for adjusted models
fill_colors = [
    'white' if '+' not in model else colors[i] 
    for i, model in enumerate(models)
]

# Create the plot
fig, ax = plt.subplots(figsize=(7, 6))

# Plot each bar with corresponding properties
for i in range(len(models)):
    ax.bar(
        i, 
        bic_values[i], 
        color=fill_colors[i], 
        edgecolor=edge_colors[i], 
        hatch=textures[i], 
        alpha=0.7, 
        linewidth=2
    )

# Set labels and title
ax.set_ylabel('BIC Values', fontsize=16)
# ax.set_title('BIC Values by Model', fontsize=16)
ax.set_xticks(np.arange(len(models)))
ax.set_xticklabels(models, rotation=45, ha="right", fontsize=12)

# Adjust y-axis limits for better visibility
y_min = min(bic_values) * 0.98
y_max = max(bic_values) * 1.02
ax.set_ylim(y_min, y_max)

# Create custom legend
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor='white', edgecolor='black', label='UniPrior Init'),
    Patch(facecolor=palette[0], edgecolor=palette[0], label='LP Init'),
    Patch(facecolor=palette[1], edgecolor=palette[1], label='Anchor Init'),
    Patch(facecolor=palette[2], edgecolor=palette[2], label='Midline Init'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='//', label='Exit Point'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='+', label='Path Sim')
]

ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

# Enhance layout
plt.tight_layout()

# Display the plot

plt.savefig("bic_values_by_model.png",dpi=300,bbox_inches='tight')
plt.show()

# Update Values

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Patch

# Data for models and corresponding r values
models = [
    "Uni Prior", "LP", "Anchor", "Midline",
    "LP+Exit Point", "Anchor+Exit Point", "Midline+Exit Point",
    "LP+Path Sim", "Anchor+Path Sim", "Midline+Path Sim"
]

r_values = [
    0.282, 0.466, 0.534, 0.437,
    0.488, 0.171, 0.383,
    0.389, 0.147, 0.109
]

# Define color palette
palette = sns.color_palette("Set2", 4)  # 4 colors for diversity
colors = [
    'black', palette[0], palette[1], palette[2],  # No Adjustment models
    palette[0], palette[1], palette[2],              # Exit Point models
    palette[0], palette[1], palette[2]               # Path Sim models
]

# Define textures for different stopping criteria
textures = [
    '', '', '', '',       # No texture for no-adjustment models
    '//', '//', '//',        # '-' texture for Exit Point models
    '+', '+', '+'      # '\' texture for Path Sim models
]

# Assign edge colors to match fill colors
edge_colors = colors.copy()

# Define fill colors: white for no-adjustment models, colored for adjusted models
fill_colors = [
    'white' if '+' not in model else colors[i] 
    for i, model in enumerate(models)
]

# Create the plot
fig, ax = plt.subplots(figsize=(7, 6))

# Plot each bar with corresponding properties
for i in range(len(models)):
    ax.bar(
        i, 
        r_values[i], 
        color=fill_colors[i], 
        edgecolor=edge_colors[i], 
        hatch=textures[i], 
        alpha=0.7, 
        linewidth=2
    )

# Add the horizontal line at the average human correlation
average_correlation = 0.28325
ax.axhline(y=average_correlation, color='darkgray', linestyle='--', linewidth=2, label=f'Avg Human Corr = {average_correlation:.3f}')

# Set labels and title
ax.set_ylabel('Correlation (r)', fontsize=16)
ax.set_xticks(np.arange(len(models)))
ax.set_xticklabels(models, rotation=45, ha="right", fontsize=12)

# Adjust y-axis limits for better visibility
y_min = min(r_values) * 0.95
y_max = max(r_values) * 1.05
ax.set_ylim(y_min, y_max)

# Create custom legend
legend_elements = [
    Patch(facecolor='white', edgecolor='black', label='UniPrior Init'),
    Patch(facecolor=palette[0], edgecolor=palette[0], label='LP Init'),
    Patch(facecolor=palette[1], edgecolor=palette[1], label='Anchor Init'),
    Patch(facecolor=palette[2], edgecolor=palette[2], label='Midline Init'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='//', label='Exit Point'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='+', label='Path Sim'),
    plt.Line2D([0], [1], color='darkgray', linestyle='--', linewidth=2, label=f'Avg Human Corr')
]

ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

# Enhance layout
plt.tight_layout()

# Save the plot
plt.savefig("correlation_values_by_model.png", dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
# if you want to include everything in the graph (not recommended)

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Patch

# Data for models and corresponding BIC values (added capped and non-capped variants)
models = [
    "Uni Prior", "LP", "Anchor", "Midline",
    "LP+Exit Point", "Anchor+Exit Point", "Midline+Exit Point",
    "LP+Exit Point (Cap)", "Anchor+Exit Point (Cap)", "Midline+Exit Point (Cap)",
    "LP+Path Sim", "Anchor+Path Sim", "Midline+Path Sim",
    "LP+Path Sim (Cap)", "Anchor+Path Sim (Cap)", "Midline+Path Sim (Cap)"
]

# Updated BIC values with capped versions
bic_values = [
    31220.49, 32375.67, 31126.21, 30212.39,           # No Adjustment
    31635.71, 30116.73, 29637.68,                     # Exit Point
    31642.99, 30119.87, 29635.80,                     # Exit Point (Capped)
    31117.88, 29891.78, 29668.85,                     # Path Sim
    30993.19, 29877.71, 29662.83                      # Path Sim (Capped)
]

# Define color palette
palette = sns.color_palette("Set2", 4)  # 4 colors for diversity
colors = [
    'black', palette[0], palette[1], palette[2],  # No Adjustment models
    palette[0], palette[1], palette[2],           # Exit Point models
    palette[0], palette[1], palette[2],           # Exit Point (Capped)
    palette[0], palette[1], palette[2],           # Path Sim models
    palette[0], palette[1], palette[2]            # Path Sim (Capped)
]

# Define textures for different stopping criteria and fill colors for capped/non-capped
textures = [
    '', '', '', '',                               # No texture for no-adjustment models
    '//', '//', '//',                             # Exit Point models
    '', '', '',                                   # Exit Point (Capped): no texture
    '+', '+', '+',                                # Path Sim models
    '', '', ''                                    # Path Sim (Capped): no texture
]

# Assign edge colors to match fill colors for non-capped, and use light grey for capped
fill_colors = [
    'white' if '+' not in model else colors[i]    # White for no-adjustment models
    for i, model in enumerate(models)
]

# For capped models, make fill color white to differentiate
for i, model in enumerate(models):
    if 'Cap' in model:
        fill_colors[i] = 'white'  # No fill color for capped models

# Create the plot
fig, ax = plt.subplots(figsize=(10, 8))

# Plot each bar with corresponding properties
for i in range(len(models)):
    ax.bar(
        i, 
        bic_values[i], 
        color=fill_colors[i], 
        edgecolor=colors[i], 
        hatch=textures[i], 
        alpha=0.7, 
        linewidth=2
    )

# Set labels and title
ax.set_ylabel('BIC Values', fontsize=16)
ax.set_xticks(np.arange(len(models)))
ax.set_xticklabels(models, rotation=45, ha="right", fontsize=12)

# Adjust y-axis limits for better visibility
y_min = min(bic_values) * 0.98
y_max = max(bic_values) * 1.02
ax.set_ylim(y_min, y_max)

# Create custom legend
legend_elements = [
    Patch(facecolor='white', edgecolor='black', label='UniPrior Init'),
    Patch(facecolor=palette[0], edgecolor=palette[0], label='LP Init'),
    Patch(facecolor=palette[1], edgecolor=palette[1], label='Anchor Init'),
    Patch(facecolor=palette[2], edgecolor=palette[2], label='Midline Init'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='//', label='Exit Point'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='+', label='Path Sim'),
    Patch(facecolor='white', edgecolor='lightgrey', label='Capped Max Steps')  # Legend for capped models
]

ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

# Enhance layout
plt.tight_layout()

# Display the plot
plt.savefig("bic_values_by_model.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Patch

# Data for models with adjustments only
models = [
    "LP+Exit Point", "LP+Exit Point (Max)", 
    "Anchor+Exit Point", "Anchor+Exit Point (Max)", 
    "Midline+Exit Point", "Midline+Exit Point (Max)", 
    "LP+Path Sim", "LP+Path Sim (Max)", 
    "Anchor+Path Sim", "Anchor+Path Sim (Max)", 
    "Midline+Path Sim", "Midline+Path Sim (Max)"
]

# BIC values for models with adjustments only
bic_values = [
    31635.71, 31642.99, 
    30116.73, 30119.87, 
    29637.68, 29635.80, 
    31117.88, 30993.19, 
    29891.78, 29877.71, 
    29668.85, 29662.83
]

# Define color palette
palette = sns.color_palette("Set2", 3)  # 3 colors for LP, Anchor, Midline
colors = [
    palette[0], palette[0],       # LP+Exit Point and LP+Exit Point (Max)
    palette[1], palette[1],       # Anchor+Exit Point and Anchor+Exit Point (Max)
    palette[2], palette[2],       # Midline+Exit Point and Midline+Exit Point (Max)
    palette[0], palette[0],       # LP+Path Sim and LP+Path Sim (Max)
    palette[1], palette[1],       # Anchor+Path Sim and Anchor+Path Sim (Max)
    palette[2], palette[2]        # Midline+Path Sim and Midline+Path Sim (Max)
]

# Use darker for non-Max and lighter for Max, with a texture for Max
fill_colors = [
    colors[i] if "Max" in models[i] else 'white'  # No fill color for Max models
    for i in range(len(models))
]

# Define hatch patterns for Max models
hatch_patterns = [
    '//', '//',   # LP+Exit Point
    '//', '//',   # Anchor+Exit Point
    '//', '//',   # Midline+Exit Point
    '+', '+',   # LP+Path Sim
    '+', '+',   # Anchor+Path Sim
    '+', '+'    # Midline+Path Sim
]

# Create the plot
fig, ax = plt.subplots(figsize=(7, 6))

# Plot each bar with corresponding properties
bar_width = 0.8  # Narrower bars to group non-max and max together
for i in range(0, len(models), 2):
    # Plot non-Max
    ax.bar(
        i, 
        bic_values[i], 
        color=fill_colors[i], 
        edgecolor=colors[i], 
        hatch=hatch_patterns[i], 
        alpha=0.7, 
        linewidth=2, 
        width=bar_width
    )
    # Plot Max next to non-Max
    ax.bar(
        i + 1, 
        bic_values[i + 1], 
        color=fill_colors[i + 1], 
        edgecolor=colors[i + 1], 
        hatch=hatch_patterns[i + 1], 
        alpha=0.7, 
        linewidth=2, 
        width=bar_width
    )

# Set labels and title
ax.set_ylabel('BIC Values', fontsize=16)
ax.set_xticks(np.arange(0, len(models), 2) + bar_width / 2)
ax.set_xticklabels([models[i] for i in range(0, len(models), 2)], rotation=45, ha="right", fontsize=12)

# Adjust y-axis limits for better visibility
y_min = min(bic_values) * 0.98
y_max = max(bic_values) * 1.02
ax.set_ylim(y_min, y_max)

# Create custom legend for non-max and max models
legend_elements = [
    Patch(facecolor=palette[0], edgecolor=palette[0], label='LP Init'),
    Patch(facecolor=palette[1], edgecolor=palette[1], label='Anchor Init'),
    Patch(facecolor=palette[2], edgecolor=palette[2], label='Midline Init'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='//', label='Exit Point'),
    Patch(facecolor='lightgrey', edgecolor='black', hatch='+', label='Path Sim'),
    Patch(facecolor='white', edgecolor='lightgrey', label='No Max Steps Limit') 
]

ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

# Enhance layout
plt.tight_layout()

# Display the plot
plt.savefig("bic_values_adjusted_models.png", dpi=300, bbox_inches='tight')
plt.show()